# 引言

最简单的神经元：房价预测

<img src="images/最简单的神经元.png" width="300"/>

为什么用神经网络

<img src="images/为什么用神经网络.png" width="300"/>    

数据集越大，对越大的神经网络的性能越好。   
对于比较小的数据集，各种模型的性能差不多

# 第一部分 ： 二分类神经网络

### （一）样品的数据结构：

一张已经被标记为“xxx”的图 被称作样品

<img src="images/三色图.png" alt="image1" width="300"/>


图像由形状$（长度，高度，深度=3）$的3D阵列表示。然而，当您读取图像作为算法的输入时，您可以将其转换为形状为$（长度*高度*3，1）$的向量。    
换句话说，可以将三维阵列“展开”或重塑为一维矢量。
也就是把3个图层的数据连起来排成一列，成为一个  $x$    
具体方法如下：

In [1]:
def image2vector(image):

    a=image.shape[0]
    b=image.shape[1]
    c=image.shape[2]
    v=image.reshape(a*b*c,1)

    return v

处理好图像后，得到向量x作为输入变量     

**输入**  
$x$: 表示一个 $n_x$ 维度特征，为输入特征，维度为 $(n_x, 1)$。


---

**输出**  
$y$: 表示输出结果，取值为 $(0,1)$;  
$(x^{(i)}, y^{(i)})$: 表示第 $i$ 组数据，可能是训练数据，也可能是测试数据，此处暂译为训练数据;

---

$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $


### （2）逻辑回归（Logistic Regression）

对于二元分类问题来说，给定一个输入特征向量 $x$，它可能属于一类或另一类，模型的任务就是找出其属于哪一类。   
我们的模型在计算过程中，需要将输入特征 $x$ 转换为输出估计值 $\hat{y}$。    
比如对于猫图而言，如果“是猫图”的 $y$ 表示为 1， “不是猫图”的 $y$ 表示为 0   
那么  $\hat{y}$  需要在（0,1）之内，表示“是猫图”的可能性



在开始之前，我们先介绍一下用sigmoid函数来处理向量x

$$ \text{For } x \in \mathbb{R}^n \text{,     } sigmoid(x) = sigmoid\begin{pmatrix}
    x_1  \\
    x_2  \\
    ...  \\
    x_n  \\
\end{pmatrix} = \begin{pmatrix}
    \frac{1}{1+e^{-x_1}}  \\
    \frac{1}{1+e^{-x_2}}  \\
    ...  \\
    \frac{1}{1+e^{-x_n}}  \\
\end{pmatrix}\tag{1} $$

sigmoid函数的导数为：
$$sigmoid\_derivative(x) = \sigma'(x) = \sigma(x) (1 - \sigma(x))\tag{2}$$

In [2]:
import numpy as np 

def sigmoid(x):
    s=1/(1+np.exp(-x))
    return s
def sigmoid_derivative(x):
    ds=sigmoid(x)*(1-sigmoid(x))    
    return ds

接下来就是预测的模型

$$
\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b), \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}
$$

Given $\{(x^{(1)}, y^{(1)}), \dots, (x^{(m)}, y^{(m)})\}$, 而我们希望 $\hat{y}^{(i)} \approx y^{(i)}$.


如何来衡量模型的准确性呢？  
需要用到 “损失函数”（loss function）：

$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$

当 y=1 时，只有 $\hat{y}$ 尽可能大（收范围限制趋近于1），损失函数L才会小   
当 y=0 时，只有 $\hat{y}$ 尽可能小（收范围限制趋近于0），损失函数L才会小

然而这只是对于一个样本的衡量方法，对于一个样本集而言，需要进行累加，这称为 成本函数（Cost Function）

$$
J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})
$$
    

即
$$
J(w, b)= \frac{1}{m} \sum_{i=1}^m \left(-y^{(i)} \log(\hat{y}^{(i)}) - (1 - y^{(i)}) \log(1 - \hat{y}^{(i)})\right)
$$


那么现在优化的方向就已经相当明确了，我们要不断修改w和b来让成本函数尽量小   
而事实上，使用了sigmoid 函数，这个成本函数是有最低点的 



<img src='images/成本函数3D.png'><img>

所以直接使用 **梯度下降法**：

假设b不变时：
$$
w = w - a \frac{dJ(w)}{dw}
$$



其中      
𝑎 ：学习率（ learning rate）    
$a \frac{dJ(w)}{dw}$ : 步长 (step），即向下走一步的长度

注意：   
对于 $\frac{dJ(w)}{dw}$  我们一般简写做 $dw$

---

拓展到两个参数就是：
$$
w := w - a \frac{\partial J(w, b)}{\partial w}, \quad b := b - a \frac{\partial J(w, b)}{\partial b}
$$


    

### （3）逻辑回归中的梯度下降（Logistic Regression Gradient Descent）

我们在前面已经了解了逻辑回归的训练过程    

那么怎么计算 $\frac{\partial J(w, b)}{\partial w}$  和  $\frac{\partial J(w, b)}{\partial b}$   呢？

假设样本有2个特征 $x_1,x_2$，那么 z 的表达式应该修改为：

$$
z=w_1x_1+w_2x_2+b
$$


回忆一下：   
$\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b) = a, \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}$  

其损失函数为：   
$L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})$    

对于单个样本而言，代价函数 $J(w,b) $ 就是损失函数：   
$L(a,y) = -ylog(a)-(1-y)log(1-a)$    

其中𝑎是逻辑回归的输出， 𝑦是样本的标签值

求导可得：   
$$ \frac{dL(a,y)}{da} = -\frac{y}{a}+\frac{1-y}{1-a} $$




同时：
$$
\begin{align*}
a &= \sigma(z) = \frac{1}{1+e^{-z}} \\
\implies 1 + e^{-z} &= \frac{1}{a} \\
\implies -e^{-z} \, dz &= -\frac{1}{a^2} \, da \\
\implies \frac{da}{dz} &= a^2 e^{-z} \\
&= a^2 \left( \frac{1}{a} - 1 \right) \\
&= a(1-a)
\end{align*}
$$


因为：   
$$ 
\frac{dL}{dz}  =\frac{dL}{da} \frac{da}{dz}     \\
$$
$$
\implies \frac{dL}{dz} =(-\frac{y}{a}+\frac{1-y}{1-a}) *  a(1-a)
$$
$$
\implies \frac{dL}{dz}= a-y
$$

同时，我们一般直接用 $dz$ 来表示 $\frac{dL}{dz}$

因为：
$$ 
\frac{dL}{dw}  =\frac{dL}{dz} \frac{dz}{dw}     \\
$$
$$
\implies \frac{dL}{dw} =(a-y)(x)
$$

所以：   
$\frac{dL}{dw_1} =(a-y)(x_1)$    

$\frac{dL}{dw_2} =(a-y)(x_2)$


因为：
$$ 
\frac{dL}{db}  =\frac{dL}{dz} \frac{dz}{db}     \\
$$
$$
\implies \frac{dL}{db} =a-y
$$

综上，对于单个样本的梯度下降算法：    
$$
w_1=w_1-adw_1  \\
w_2=w_2-adw_2   \\
b=b-adb
$$
其中
$$
dw_1 =(a-y)(x_1) \\
dw_2 =(a-y)(x_2)\\
db=a-y\\

$$

上述结论拓展到m个样本应该如何呢？     
我们知道：
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})  $     
即：  
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(a^{(i)}, y^{(i)})$    
其中 $a^{(i)}$ 是第 i 个样本的预测输出，$y^{(i)}$是第 i 个样本的实际标签。








对权重 \(w_1\) 的梯度：   
$$
\frac{\partial J}{\partial w_1} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_1^{(i)}
$$

对权重 \(w_2\) 的梯度：
$$
\frac{\partial J}{\partial w_2} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_2^{(i)}
$$

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$

梯度下降更新规则
使用上述计算出的平均梯度来更新参数：
$$
w_1 = w_1 - \alpha \frac{\partial J}{\partial w_1} \\
w_2 = w_2 - \alpha \frac{\partial J}{\partial w_2} \\
b = b - \alpha \frac{\partial J}{\partial b}
$$

其中，$\alpha$ 是学习率，用于控制更新步骤的大小。

### （四）向量化（Vectorization）

在上面的计算方法中,每进行一次梯度下降时，都需要使用for循环来遍历每一个样本    
事实上这样的效率非常低，所以我们学习使用 **向量化** 来解决这个问题

所谓向量化，就是对于一串数据，比如数组或者矩阵，本来采用for循环进行一个个的运算，    
但是现在使用numpy库的自带功能，去掉显式的for循环     


In [3]:
import numpy as np
import time

data = np.random.rand(1000000)  # 假设有一个大型数组

# 向量化计算平方根和平均值
tik = time.time()
sqrt_data_vectorized = np.sqrt(data)
tok = time.time()
time1 = tok - tik
print('time:', time1)
print(sqrt_data_vectorized[:5])  # 打印前5个元素作为示例


# 普通计算方法
tik = time.time()
sqrt_list = []  # 使用列表收集平方根结果
for i in data:
    sqrt_list.append(i**0.5)  # 向列表追加平方根
sqrt_data = np.array(sqrt_list)  # 将列表转换为NumPy数组
tok = time.time()
time2 = tok - tik    
print('time:', time2)
print(sqrt_data[:5])  # 打印前5个元素作为示例


time: 0.0009965896606445312
[0.58690181 0.66052454 0.98972112 0.64179344 0.67875717]
time: 0.13191938400268555
[0.58690181 0.66052454 0.98972112 0.64179344 0.67875717]


上面的结果可以看到相差了100多倍    
注意：  
所谓向量化，不一定要调用numpy的计算函数比如 np.mean() np.sqrt()等，而是使用了np.array 的数据格式   
比如对于一个 np.array 进行 **2 平方操作，也是向量化的

根据我们已经学会的算法，可以知道现在的计算过程如下：   

初始化 $J=0$, $dw_1=0$, $dw_2=0$, $db=0$。代码流程如下：

```python
J = 0; dw1 = 0; dw2 = 0; db = 0
for i in range(1, m+1):  # 假设m是样本数量
    z_i = w * x[i] + b  # 这里假设x[i]是一个包含x1和x2的向量
    a_i = sigmoid(z_i)
    J += -[y[i] * log(a_i) + (1 - y[i]) * log(1 - a_i)]
    dz_i = a_i - y[i]
    dw1 += x[i][0] * dz_i  # 假设x[i][0]是特征x1
    dw2 += x[i][1] * dz_i  # 假设x[i][1]是特征x2
    db += dz_i

# 外部循环结束后，计算平均值
J /= m
dw1 /= m
dw2 /= m
db /= m

# 更新参数
w1 = w1 - alpha * dw1
w2 = w2 - alpha * dw2
b = b - alpha * db


上面这段代码中实现了一次梯度下降，也就是一次训练，但是使用了两个循环     
第一个循环是for循环遍历每一个样本     
第二个循环是对特征值进行循环。在这例子我们有 2 个特征值。如果你有超过两个特征时，需要循环 𝑑𝑤1 、 𝑑𝑤2 、 𝑑𝑤3 等等。



我们先来看**第二个循环的向量化**      
不用初始化 𝑑𝑤1 𝑑𝑤2 都等于 0，而是定义 𝑑𝑤 为一个向量，设置 $ w=np.zeros(n_x,1)  $  定义一个$n_x$行的一维向量    
其中 $n_x $代表单个样本的特征数，对于一张图片而言可能是64* 64 *3    
而其实dw不需要专门在代码开头初始化，因为dw是根据w自动算出来的，形状也是取决于w

<img src="images/第一层向量化.png">

再来看 **第一个循环的向量化**     
回忆一下最开始讲到的输入集


$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$n_x$ 是单个样本的特征点数，对于图片而言可能是64* 64 *3

$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $



所以先计算 $z_1,z_2,z_3, \dots, z_n$  ,把他们都放到一个  $1 \times m$ 的行向量中    
你可以发现他可以表达为  $w^T$  (w的转置) $\times X+[b,b,\dots,b]$    
$[b,b,\dots,b]$ 是一个 $1 \times m$ 的行向量

所以计算的最终得到的$Z$是一个 $1 \times m$ 的向量，$Z = \begin{bmatrix} z^{(1)} & z^{(2)} & \ldots & z^{(m)} \end{bmatrix}$，计算方式为 $z = w^T X + \mathbf{b}$，其中 $\mathbf{b} = \begin{bmatrix} b & b & \ldots & b \end{bmatrix}$   

$$
Z = \begin{bmatrix}
w^T x^{(1)} + b \\
w^T x^{(2)} + b \\
\vdots \\
w^T x^{(m)} + b
\end{bmatrix}
$$

其中，
- $w^T x^{(1)} + b$ 是向量 $Z$ 的第一个元素，
- $w^T x^{(2)} + b$ 是第二个元素，
- 以此类推，直到 $w^T x^{(m)} + b$ 是第 $m$ 个元素。



In [ ]:
Z=np.dot(w.T, X)+b
# 其中b通过广播机制自动被拓展成一个 $1 \times m$ 的行向量

加下来要使用向量Z计算出向量Y    
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

然后就可以计算 $ dZ =A-Y=[a^{(1)}-y^{(1)},a^{(2)}-y^{(2)},\dots, a^{(n)}-y^{(n)}   ]  $

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$
所以 $ db=\frac{1}{m}*\sum_{i=1}^m (dz^{(i)}) $   


In [ ]:
db=(1/m)*np.sum(dZ)

对权重 \(w\) 的梯度：
$$
\frac{\partial J}{\partial w} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x^{(i)}
$$

所以  $ dw=\frac{1}{m}*X*dz^T $   

### (五)总结

这是没有使用向量化的计算过程

<img src='images/原始计算过程.png'>

这是使用向量化之后的计算过程   


<img src='images/两层向量化.png'>


### (六)注意事项

这里要讲一个重要且常见的bug——一维数组

In [ ]:
import numpy as np

a=np.random.rand(5)
print(a.shape)  #(5,)
# 此时a就是一个一维数组，既不是行向量也不是列向量

这种结构很容易出现意想不到的bug，所以是要坚决摈弃的，一定要修改成(5,1)

In [ ]:
import numpy as np

a=np.random.rand(5,1)
print(a.shape)  #(5,1)

# 第二部分：浅层神经网络（Shallow neural networks）

### （一）浅层神经网络介绍

在上一部分，我们学习了二分类神经网络     
<img src='images/二分类神经网络.png'>

中间这个节点的计算过程如下：

$$
\begin{array}{c}
\textbf{x} \\
\textbf{w} \\
b \\
\end{array}
\rightarrow z = w^T x + b    \\
\rightarrow \alpha = \sigma(z)   \\
$$


这里我们将要学习浅层的神经网络   
<img src='images/浅层神经网络.png'>    




其中，我们用方框[1]表示第一层, [2]表示第二层，以此类推     
其中 [0] 也称为输入层，最后的 [2] 也称为输出层，中间的都叫做隐藏层。      
输入层是不算的，所以这一共有两层，所以叫“双层神经网络”    
在浅层的神经网络这幅图中，每一个神经节点也都是执行相同的操作      

以第一层的第一个节点为例
$$
\begin{array}{c}
\textbf{x} \\
W^{[1]} \\
b^{[1]} \\
\end{array}
\rightarrow z^{[1]} = W^{[1]} x + b^{[1]} \rightarrow a^{[1]} = \sigma(z^{[1]})
$$




但是正如图中所见，一层中是有多个神经元的    

<img src='images/双层神经网络.png'>


每一个节点最终都会输出一个预测值（或者叫激活值）a，就是我们在第一部分中的a    （ a 表示激活的意思）   
为了普遍化，我们直接把每个节点（包括输入层和输出层）的最终值都称为 a      
输入层的激活值称为 $a^{[0]}$，第一层的激活值记作 $a^{[1]}$。   
第一层的第一个单元或结点我们将其表示为 $a_1^{[1]}$，第二个结点的值我们记为 $a_2^{[1]}$ 以此类推。


$$
a^{[1]} = \begin{bmatrix}
a_1^{[1]} \\
a_2^{[1]} \\
a_3^{[1]} \\
a_4^{[1]}
\end{bmatrix}
$$


出于显而易见的向量化考虑，可以让$a^{[1]}$成为一个 $4 \times 1$ 的矩阵，$a^{[2]}$成为一个 $1 \times 1$ 的矩阵    
注意：   
**图中的 $x_1,x_2,x_3$ 等是指一个样本的不同特征,一共有$n_x$个，这里简化表达为3个，对于一张图片而言可能$n_x=64*64*3$**

同样的道理，每层都有每层的W参数和b参数，    

显而易见 $W^{[1]}$ 的形状是 $4 \times 3$,  $b^{[1]}$ 的形状是 $4 \times 1$    
（4是因为本层有4个节点，每个节点都有一个w，3是因为每个w都要处理3个样本特征，所以是3）   

同理可得 $W^{[2]}$ 的形状是 $1 \times 4$,  $b^{[2]}$ 的形状是 $1 \times 1$    
（1是因为本层有1个节点，每个节点都有一个w，4是因为每个w都要处理4个上一隐藏层的a的输入，所以是4）


### （二）浅层神经网络的计算

模仿二分类神经网络的计算，浅层神经网络的计算过程如下：       
每个小圆圈代表了计算的两个主要概念。

第一步，计算 $z_1^{[1]} = w_1^{[1]T} x + b_1^{[1]}$。

第二步，通过激活函数计算 $a_1^{[1]} = \sigma(z_1^{[1]})$。

随着层的第一个以及后面的每个神经元的计算过程一样，只是注意每层表示不同，层级分别对应 $a_2^{[1]}, a_3^{[1]}, a_4^{[1]}$，详细过程见下：

$$
\begin{align*}
z_1^{[1]} &= w_1^{[1]T} x + b_1^{[1]}, & a_1^{[1]} &= \sigma(z_1^{[1]}) \\
z_2^{[1]} &= w_2^{[1]T} x + b_2^{[1]}, & a_2^{[1]} &= \sigma(z_2^{[1]}) \\
z_3^{[1]} &= w_3^{[1]T} x + b_3^{[1]}, & a_3^{[1]} &= \sigma(z_3^{[1]}) \\
z_4^{[1]} &= w_4^{[1]T} x + b_4^{[1]}, & a_4^{[1]} &= \sigma(z_4^{[1]})
\end{align*}
$$


显而易见地可以使用向量化计算     
$$
Z^{[1]} = W^{[1]T} x + b^{[1]},  a^{[1]} = \sigma(Z^{[1]})   \\
\\
Z^{[n]} = W^{[n]T} x + b^{[n]},  a^{[n]} = \sigma(Z^{[n]})
$$

<img src='images/双层计算过程1.png'>

拓展到整个过程，就是如图    
<img src='images/双层计算过程2.png'>

同时，由于目前所有的计算都是针对单个样本的多个特征进行，所以还需要多个样本进行向量化      

**注：** 例如 $a^{[2](i)}$  中 (i) 是第i个训练样本，[2]是第二层

对于所有训练样本，需要让 𝑖 从 1到 𝑚实现下面这四个等式：

**计算第一层神经元的输出**：
- 计算线性组合 ：
$$
z^{[1](i)} = W^{[1]}(i) \cdot x^{(i)} + b^{[1]}(i)
$$
- 应用激活函数：
$$
a^{[1](i)} = \sigma(z^{[1](i)})
$$

**计算第二层神经元的输出**：
- 计算线性组合：
$$
z^{[2](i)} = W^{[2]}(i) \cdot a^{[1](i)} + b^{[2]}(i)
$$
- 应用激活函数：
$$
a^{[2](i)} = \sigma(z^{[2](i)})
$$



所以向量化之后总结如下：    

<img src='images/双层向量化.png'>



### （三）激活函数

sigmoid函数有时候并不是最好的激活函数。这里介绍4种激活函数

注：下面提到的“梯度消失”指x值很大的时候斜率很小


#### 1. Sigmoid 函数
- **公式**：$ \sigma(z) = \frac{1}{1 + e^{-z}} $
- **求导**：$ \sigma'(z) = \sigma(z)(1 - \sigma(z)) $
- **图像**：   
  <img src="images/sigmoid函数.png" width="300"/>
- **使用场景**：
  - 通常用于二分类问题中的输出层，因为它的输出范围是 (0,1)，可以表示概率。
  - 不太适用于隐藏层，因为在深层网络中容易引起梯度消失的问题。

#### 2. Tanh 函数
- **公式**：$ \tanh(z) = \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}} $
- **求导**：$ \tanh'(z) = 1 - \tanh^2(z) $
- **图像**：  
  <img src="images/tanh函数.png" width="300"/>
- **使用场景**：
  - 常用于隐藏层，因为它的输出范围是 (-1,1)，比 sigmoid 函数的输出范围更广，可以更好地帮助模型学习。
  - 同样存在梯度消失的问题，尤其是在深层网络中。

#### 3. ReLU 函数
- **公式**：$ \text{ReLU}(z) = \max(0, z) $
- **求导**：
  - 当 $ z > 0 $ 时, $\text{ReLU}'(z) = 1$
  - 当 $ z \leq 0 $ 时, $\text{ReLU}'(z) = 0$
- **图像**：   
  <img src="images/ReLU函数.png" width="300"/>
- **使用场景**：
  - 非常流行用于各种网络的隐藏层，特别是在卷积神经网络中。
  - 有助于解决梯度消失问题，训练速度通常比 sigmoid 和 tanh 快。
  - 但存在死神经元问题，即一旦输入小于0，ReLU 激活的导数为0，该神经元可能再也不会对任何数据有激活现象了。

#### 4. Leaky ReLU 函数
- **公式**：$ \text{Leaky ReLU}(z) = \max(0.01z, z) $
- **求导**：
  - 当 $ z > 0 $ 时, $\text{Leaky ReLU}'(z) = 1$
  - 当 $ z \leq 0 $ 时, $\text{Leaky ReLU}'(z) = 0.01$
- **图像**：   
  <img src="images/Leaky ReLU函数.png" width="300"/>
- **使用场景**：
  - 用于解决 ReLU 死神经元问题，允许小的梯度当 \( z \leq 0 \) 时通过，提供所有神经元持续的梯度更新。
  - 适合用于深度学习网络中，特别是在面对复杂问题时，能够提高模型的稳定性。



### （四）浅层神经网络的梯度下降

我们用$n_x$表示输入特征的个数，所以根据不同的层数,$n^{[1]}$表示隐藏层单元个数，$n^{[2]}$表示输出单元个数

**神经网络权重和偏置的维度说明：**

- 第一层的权重 $ W^{[1]} $ 和偏置 $ b^{[1]} $ 的维度分别为 $ (n^{[1]}, n^{[0]}) $ 和 $ (n^{[1]}, 1) $，其中 $ n^{[0]} $ 是输入层的特征数量，$ n^{[1]} $ 是第一层的单元数或神经元数。
- 第二层的权重 $ W^{[2]} $ 和偏置 $ b^{[2]} $ 的维度分别为 $ (n^{[2]}, n^{[1]}) $ 和 $ (n^{[2]}, 1) $，其中 $ n^{[2]} $ 是第二层的单元数。

**loss function**：（与第一部分完全相同）

$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$



**Cost function:**

$$
J(W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}, y)
$$




**Forward Propagation 正向传播:**


   $$
   z^{[1]} = W^{[1]} x + b^{[1]}
   $$

   $$
   a^{[1]} = \sigma(z^{[1]})
   $$

   $$
   z^{[2]} = W^{[2]} a^{[1]} + b^{[2]}
   $$

   $$
   a^{[2]} = g(z^{[2]}) = \sigma(z^{[2]})
   $$


**Back Propagation 反向传播:**


   $$
 \quad dz^{[2]} = a^{[2]} - Y, \quad Y = [y^{[1]} \dots y^{[m]}]
   $$

   $$
 \quad dW^{[2]} = \frac{1}{m} dz^{[2]} (a^{[1]})^T
   $$

   $$
 \quad db^{[2]} = \frac{1}{m} \text{np.sum}(dz^{[2]}, \text{axis} = 1, \text{keepdims} = \text{True})
   $$

   $$
 \quad dz^{[1]} = W^{[2]T} dz^{[2]} * g^{[1]'}(z^{[1]})
   $$

   $$
 \quad dW^{[1]} = \frac{1}{m} dz^{[1]} x^T
   $$


   $$
 \quad db^{[1]} = \frac{1}{m} \text{np.sum}(dz^{[1]}, \text{axis} = 1, \text{keepdims} = \text{True})
   $$

注意：
1. $g^{[1]'}$ 表示在隐藏层中使用的激活函数的导数。
2. axis=1表示水平相加求和
3. keepdims=True 是为了防止出现(n,)这种，确保输出为(n,1),或者使用reshape

**梯度下降：**    


$$
W^{[1]} = W^{[1]} - \alpha dW^{[1]}
$$

$$
b^{[1]} = b^{[1]} - \alpha db^{[1]}
$$

$$
W^{[2]} = W^{[2]} - \alpha dW^{[2]}
$$

$$
b^{[2]} = b^{[2]} - \alpha db^{[2]}
$$






**注意：这里只给出了公式，没有给出推导，没必要**


### （五）随机初始化

与之前的二分类神经网络不同，多层神经网络的权重W不能都是0    
因为如果 $W^{[1]}$ 和 $W^{[2]}$ 都是全为0的矩阵的话，就会导致这个神经网络每一个节点完全相同，完全对称，这就失去了神经网络的意义     
所以要对W进行高斯分布的随机初始化    
然而b是不需要的


<img src='images/随机初始化.png'>

In [ ]:
#对于上面这个神经网络而言，需要随机初始化如下
W_1=np.random.randn(2,2)*0.01
W_2=np.random.randn(2,2)*0.01
b_1=np.zeros((2,1))
b_2=0

为什么要乘以0.01呢？    
因为让权重小一点，对于隐藏层激活函数是tanh或炸sigmoid的，训练速度会快很多。   
因为中间斜率大

### (六)注意事项

1. **W1，W2等是不需要再向量化到一个矩阵中的**   
向量化的意思是：W1内部的多个同层节点的计算方法相同，所以可以保存到一个大矩阵W1中，同理推广     
然而对于W1和W2而言，他们作为权重，具有不同的形状，不同的激活函数，不同的正反向传播算法，所以是不可以放到一个向量中的     
对于n1，n2,还有b1,b2也是这个道理

2. **关于训练集和测试集**：    
注意训练集和测试集不能只包含猫，否则会导致它无法学习如何区分猫和非猫。这意味着你的模型实际上没有进行有效的学习来解决二分类问题，而只是学会了识别所有输入为同一类别。
在这种情况下，会导致输出的准确率变成100%

3. **关于隐藏层节点数的选择**：   
对于较简单的问题，较少的单元可能就足够了。   
对于复杂的问题，如复杂的图像识别或语音识别任务，可能需要更多的隐藏层和更多的单元     
过拟合：如果隐藏层单元太多，模型可能会过度学习训练数据的细节和噪声，导致在新的或未见过的数据上表现不佳。    
欠拟合：如果隐藏层单元太少，模型可能没有足够的能力来捕捉数据中的复杂模式，导致训练和测试性能都不佳。    
通常需要通过实验来找到合适的层数和每层的单元数。可以开始于一个相对较小的网络，逐渐增加单元数或层数，直到测试误差不再显著下降。   
还有一些经验法则如：   
a. 隐藏层单元数可以设置为输入层和输出层单元数的平均值。   
b. 可以尝试设置隐藏层单元数为输入特征数的2/3加上输出单元数。   


# 第三部分：深层神经网络（Deep Neural Networks） 

### （一）深层神经网络的介绍

<img src='images/四层神经网络.png'>

上图中我们看到一个典型的四层神经网络，对于每一层的节点数，可以这样表示：    
 
$$L=4 ， n^{[0]}=n^x=3 , n^{[1]}=5 , n^{[2]} =5 , n^{[3]}=3 ， n^{[4]}=n^{[L]}=1 $$

每一层l的激活函数记为 $a^{[l]}$, 其中第0层的$X=a^{[0]}$

下图解释了深度神经网络的内在机制，浅层隐藏层会去识别边缘，中层隐藏层识别部分特征，深层隐藏层可以构建一张完整人脸    

<img src='images/深层神经网络原理解释.png' width=500>

这种原理对于语言识别等神经网络也是适用的，比如浅层神经元处理小的音调升降，中间神经元识别音位（比如元音辅音），深层神经元识别单词，再形成词组，句子等

另外，相比于浅层神经网络，想要得到相同的计算结果，深层神经网络需要的总节点数量 比浅层神经网络 要少非常非常多（尤其是对于规模比较大的计算）

### (二)深度神经网络的正向和反向传播

   
对于l层而言：    

**正向传播：**

   
   $$
   Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}
   $$

   $$
   A^{[l]} = g^{[l]}(Z^{[l]})
   $$

其中对于第一层而言$A^{[l-1]}=A^{[0]}$就是$X$    

**反向传播：**
$$ 
dz^{[l]} = da^{[l]} \ast g'^{[l]}(z^{[l]})  \\
dw^{[l]} = dz^{[l]} \cdot a^{[l-1]}  \\
db^{[l]} = dz^{[l]}  \\
da^{[l-1]} = (w^{[l]})^T \cdot dz^{[l]}  \\
$$
向量化之后如下：
$$
dZ^{[l]} = dA^{[l]} \ast g'^{[l]}(Z^{[l]})   \\
dW^{[l]} = \frac{1}{m} dZ^{[l]} \cdot (A^{[l-1]})^T    \\
db^{[l]} = \frac{1}{m} \text{np.sum}(dZ^{[l]}, \text{axis} = 1, \text{keepdims} = True)  \\
dA^{[l-1]} = (W^{[l]})^T \cdot dz^{[l]}   \\
$$

整个计算流程如图

<img src='images/深层神经网络计算机制.png' width=600>

对于每一次训练，从X开始，向右正向传播，计算成本函数，然后反向传播    
从计算过程中可以看出，反向传播中需要正向传播的相应环节传递参数$Z^{[l]}$

但是仔细观察参数，我们可以发现有一个参数似乎没有得到计算，反向传播的第一个$dA^{[L]}$     
我们必须先算出这个值，   由于对于最后一层而言，     
$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$
其中$y$是真实标签，$\hat y$ 是预测概率，也就是$A^{[L]}$   
所以要求的$dA^{[L]}$其实就是


$$
 dA^{[L]}=\frac{\partial L}{\partial A^{[L]}} = - \left( \frac{y}{A^{[L]}} - \frac{1 - y}{1 - A^{[L]}} \right) 
$$

用python表达式就是


In [ ]:
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

每一层的维度：   
$$
b^{[l]} : (n^{[l]},1)   \\
W^{[l]} : (n^{[l]},n^{[l-1]})  \\

$$

$dW^{[l]}$和$W^{[l]}$的维度相同，$db^{[l]}$和$b^{[l]}$的维度相同

对于z和a而言：    
**向量化前：**    
$$
Z^{[l]} : (n^{[l]},1)   \\
A^{[l]} : (n^{[l]},1)   \\
$$
**向量化后**:

$$
Z^{[l]} = (z^{[l](1)}, z^{ }, z^{ }, \ldots, z^{[l](m)})   \\
Z^{[l]} : (n^{[l]}, m)  \\
A^{[l]}: (n^{[l]}, m)    \\
A^{[0]} = X = (n^{[0]}, m)
$$


### （三）注意事项

1. 不是只有第一个W和b需要更新，每一个W和b都要更新！这样才是每一个神经元都得到训练！
2. 对于深度神经网络，在初始化W的时候，不要再使用*0.01来初始化权重W的标准差，这会导致神经网络在训练初期就陷入较小的梯度值（梯度消失问题）， cost会一直收敛在0.693147，也就是-log(0.5)。对于ReLU激活函数，建议使用He初始化。    He 初始化有助于神经网络更快地收敛。这是因为它避免了权重在训练初期过小或过大，这些极端值可能会导致学习过程缓慢或不稳定。
3. 对于深度神经网络，仅仅是100张照片是远远不够的，虽然理论上可以用很少的图片（如几百张）来训练一个模型，但这通常会导致模型性能不佳或者过拟合。有些来源建议至少需要1000张图片每个类别来训练一个稳健的模型 。数据的多样性和质量同样重要。不仅需要数量足够，还需要保证数据能够覆盖到类别的各种变化，包括不同的环境、角度、光照条件等。
4. 0.6931471就是没有成功学习的表现


# 第四部分：深层神经网络的优化

### (一) 训练集，验证集和测试集(train\Dev\test sets)

一般把收集到的数据按照 6 : 2 ：2的比例分配。但是对于非常大体量的数据而言，验证集和测试集占比例会更小。    
同时，应当尽可能保持 验证集和测试集 相似，防止出现问题    

如果数据量不够，没有测试集也没有关系


### （二）偏差和方差（Bias\Variance）

欠拟合（underfitting）：高偏差（high bias）    
过拟合（overfitting）： 高方差（high variance）     
适度拟合（just right）

如何判断是哪种情况呢？    
假设最优误差是0% （最优误差又称为贝叶斯误差，是人眼识别的误差）   

| 情况         | 训练集误差 | 测试集误差 | 描述         |
|--------------|------------|------------|--------------|
| 过拟合（高方差） | 1%         | 15%        | 过拟合，模型在训练集上表现很好但在测试集上表现较差，说明模型过于复杂，对训练数据过度敏感。 |
| 欠拟合（高偏差） | 15%        | 16%        | 欠拟合，模型在训练集和测试集上都表现不佳，说明模型过于简单，无法捕捉基本趋势。 |
| 高偏差+高方差  | 15%        | 30%        | 模型在训练集上的误差较高，并且在测试集上误差更大，说明模型既不能准确捕捉趋势也无法泛化到新数据。 |
| 适度拟合      | 1%         | 2%         | 适度拟合，模型在训练集和测试集上都表现很好，误差低且相差不大，说明模型泛化能力强。 |


总之：训练集的误差决定是否是 高偏差，测试集和训练集的准确率差异大小决定是否是 高方差    
注意：如果最优误差不是0%，而是比如15%，那么这里的第二种情况也可以算作是 适度拟合


方差和偏差是在优化算法中最基本的衡量指标：   


第一步：判断是否高偏差（是否成功拟合）
- **如果偏差偏高：**
  1. 使用更深更复杂的网络
  2. 增加训练时间
  3. 使用更先进的优化算法
  4. 使用NN卷积神经网络

- **当偏差正常之后，进行第二步：**

第二步: 判断是否高方差（是否过拟合）
- **如果方差偏高：**
  1. 增加数据量
  2. 正则化（Regularization）
  3. 使用NN卷积神经网络


### （三）正则化 Regulation

正则化主要有两种：**L2正则化 和 dropout机制**  
其实还有L1正则化，但是性能不如L2所以就不介绍了

#### **L2正则化**

对于单个节点而言，之前我们对成本函数的定义如下：    
$$
J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})
$$

现在修改如下：
$$
J(w, b) = \frac{1}{m} \sum_{i=1}^{m} L({\hat y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} \|w\|_2^2 \\
||w||_2^2 = \sum_{j=1}^{n_x} w_j^2 = w^T w
$$


注意： 
1. $||w||_2$是向量参数w的欧几里得范数（也就是L2范数），算法如上。   $||w||_1$是L1范数，算法有点不一样，这里不讲了
2. $\lambda$ 是正则化参数，需要慢慢试出来，也是一个超参数



神经网络中的L2正则化：    
$$
J(w^{(1)}, b^{(1)}, \ldots, w^{(L)}, b^{(L)}) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} \sum_{l=1}^L \|w^{(l)}\|_F^2 \\
\|w^{(l)}\|_F^2 = \sum_{i=1}^{n^{(l-1)}} \sum_{j=1}^{n^{(l)}} (w_{ij}^{(l)})^2
$$


注意：   
1. 之前讲过 $ W^{[l]} $的形状是 $(n^{[l]},n^{[l-1]})  $
2. 该矩阵范数被称为弗罗贝尼乌斯范数，用下标F表示 


对于L2正则化，其反向传播（BackProp）如下：     
$$
\frac{\partial J}{\partial w} = \frac{\partial L}{\partial w} + \frac{\lambda}{m} w \\
dW = \text{From backprop} + \frac{\lambda}{m} w \\
w := w - \alpha dW
$$


注意:    
1. 显而易见，W被减去的更多了，所以L2正则化又叫 **权重衰减（Weight Decay）**
2. From backprop 是通过反向传播算法计算得到的权重的梯度,也就是上一部分中已经算出的部分
3. $\lambda$ 的取值方法：    
- **小型模型和/或大型数据集**：尝试 $10^{-7}$ 到 $10^{-5}$
- **大型模型和/或小型数据集**：尝试 $10^{-5}$ 到 $10^{-3}$


#### **Dropout正则化**



dropout会遍历网络的每一层，并随机从这一层中消除某几个节点。每一层都要设置一个保留节点的概率（比例），各个层设置的比例可以不一样      
这种随机“关闭”神经元的效果是，网络不能依赖于任何一个神经元，因为它可能在任何训练轮次中被关闭。     
这迫使网络分散学习信息，从而增加其泛化能力。

如何实现dropout呢？我们以一个层数L=3的神经网络为例    
首先要设计一个具体数字keep_prob,大小为0~1，表示保留某个隐藏单元的概率。      
下面的案例中被设置为0.8，表示为每个节点有0.8的概率被保存    
然后定义d，$d^{[3]}$表示一个3层的dropout向量    

In [ ]:
keep_prob=0.8
d3=np.random.rand(a3.shape[0],a3.shape[1])< keep_prob
# d3 是一个与 a3 （某层的激活输出）同形状的布尔矩阵，其中的每个元素都是通过比较一个随机数（从均匀分布中抽取）和 keep_prob 获得的。
# 如果随机数小于 keep_prob，相应位置的值为 True（意味着该神经元在这次迭代中保持激活），否则为 False（神经元输出被置为0）。

接下来更新a3，直接把a3和d3相乘，让$d^{[3]}$中的为0的元素和$a^{[3]}$中的对应元素归零

In [ ]:
a3=np.multiply(a3,d3)
#python会自动把True和False转化为1和0

a3/=keep_prob  # 最后向外拓展a3

为什么要除keep_prob呢？       
因为a3中已经有20%的节点被删除了，为了不影响Z4的期望值，a3要除以0.8来修正或弥补所需的那20%, 使Z4的期望值不变


dropout机制一般在正向传播中使用，并且每训练一次就要dropout一次     
但是在对应的反方向传播的每一层也要使用 对应相同的dropout mask  进行处理 

注意：    
1. 当keep_prob被设置为1时，就关闭了dropout机制
2. 在测试阶段，不需要使用dropout
3. 对于节点数比较多的层，把keep_prob设置的小一点（多删一点），反之设置的大一点
4. 除非算法过拟合，否则不使用 dropout


dropout机制的一大缺点是代价函数J不能被再明确定义

#### **正则化的意义**

L2正则化使部分权重被衰减，Dropout机制直接删除了部分节点，所以都是简化了网络，但是缺没有减少深度，所以防止了过拟合

同时，由于sigmoid和tanh函数中间部分更接近于线性。而当W变小时，Z也会变小，所以会更接近函数的中间部分，也就是接近线性。    
而线性层的叠加是不会增加复杂度的

### （四）其他正则化方法

除了上面说到的L2正则化和Dropout之外，还有几种正则化方法

方法一：人工图片扩增


<img src='images/人工图片扩增.png'>

方法二：**Early Stopping**

<img src='images/early stopping.png' width=500>    

上图中蓝线代表训练集误差，紫线代表测试误差    
通过及时停止训练，来防止过拟合


但是这种方法有一个缺点：
就是不能独立地处理这两个问题，因为提早停止梯度下降，也就是停止了优化代价函数J，    
因为现在你不再尝试降低代价函数，所以代价函数J的值可能不够小，同时你又希望不出现过拟合，    
你没有采取不同的方式来解决这两个问题，而是用一种方法同时解决两个问题，这样做的结果是我要考虑的东西变得更复杂。

Early stopping的优点是，只运行一次梯度下降，你可以找出 𝑤的较小值，中间值和较大值，而无需尝试 𝐿2正则化超级参数 𝜆的很多值。

### （五） 归一化输入 （Normalizing inputs）

这个方法用于加速训练

假设一个训练集有两个特征（输入特征为2维），归一化需要两个步骤：
1. 把均值降到0
2. 归一化方差



<img src='images/归一化输入.png' width=500>

归一化的目的是将输入数据转换为统一的尺度，帮助优化算法（如梯度下降）更快地收敛到最小损失


$$
X_{\text{std}} = \frac{X - \mu}{\sigma}
$$

其中 $\mu$ 是特征的均值，$\sigma $ 是特征的标准差。  

$$
\mu = \frac{1}{m} \sum_{i=1}^m x_i
$$




$$
\sigma = \sqrt{\frac{1}{m} \sum_{i=1}^m (x_i - \mu)^2}
$$




注意：
1. 其中 m 是特征值的数量，$x_i$ 是每个特征值。     
2. 这种转换后，特征集 X 的新均值将是0，标准差将是1，即 $X_{\text{std}}$ 的所有值都围绕0分布，具有单位方差。
3. 要用同样的方法调整测试集，而不是在训练集和测试集上分别预估 𝜇 和 𝜎2。  
    也就是说测试集使用的 𝜇和 𝜎2 也是由训练集数据计算得来的。



为什么要归一化输入呢？    
可以看下图中的代价函数3维图形（注：这些数据轴应该是 𝑤1和 𝑤2）


<img src='images/为什么要归一化输入.png' width=500 >

如上图所示，优化前必须要使用比较小的学习率（步长），但是优化后成本函数更圆，所以可以使用更大的学习率

注意：
1. 这里的更快不是指计算更快。而是指可以使用更大的学习率，更快达到最优点
2. 当 x1,x2,x3 的大小范围比较相似时，用不用归一化其实差别不大。但是如果大小范围差别很大，归一化特征值就非常重要了。
3. 执行这类归一化并不会产生什么危害，所以可以都加上

### （六）梯度消失/梯度爆炸（Vanishing/Exploding gradients）

在训练深度神经网络时，有时坡度会变得非常大或者非常小

梯度爆炸是因为在极深的神经网络中，某一个位置的权重经过反复的乘法，出现了指数爆炸的现象    
梯度消失的原理也是一样的

要解决这个问题，需要研究一下 **神经网络的权重初始化**

在神经网络中，线性组合的计算公式为：
$$
Z = w_1x_1 + w_2x_2 + \ldots + w_nx_n，b = 0，
$$
暂时忽略 𝑏，为了预防 𝑧值过大或过小， 所以当𝑛越大时，我们希望 𝑤𝑖越小，
所以权重$w_i$的初始化公式为：
$$
w_i = \frac{1}{n}
$$
其中，$n$是神经元的输入数量。

在Python中，可以使用numpy库如下初始化第$[l]$层的权重：
$$
W[l] = np.random.randn(shape) * np.sqrt(\frac{1}{n^{[l-1]}})
$$

**He初始化**     
适用于使用RuLU激活函数的层

In [ ]:
parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2. / layer_dims[l-1])  #He初始化

**Xavier初始化**     
适用于使用tanh激活函数的层

In [ ]:
parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(1. / layer_dims[l-1])

# 第五部分：算法的优化 （Optimization Algorithms）

### （一）Mini-batch 梯度下降（Mini-batch gradient descent）

我们之前尝试过，对于大量的训练数据，进行一次梯度下降的时间非常长。   

$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$ 。   维度 $n_x \times m$     
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$ 。   维度 $1 \times m$     


现在把 X 分为很多个小集合，称为mini-batch。    
比如把 $x^{(1)}到 x^{(100)}$ 作为一个mini-batch， 称为 $ X^{\{1\}}$,     
那么如果 $X = [x^{(1)}, x^{(2)}, \dots, x^{(1000)}]$，则就可以拆分为$ X^{\{1\}} $ 到 $ X^{\{10\}}$     
同理，如果 $Y = [y^{(1)}, y^{(2)}, \dots, y^{(1000)}]$，则就可以拆分为$ Y^{\{1\}} $ 到 $ Y^{\{10\}}$   

所以 $ X^{\{t\}}$ 的大小为 $n_x \times n$ ,  $ Y^{\{t\}}$ 的大小为 $1 \times n$，    
其中n为单个 mini-batch 的样本数 

注意：    
使用小括号上标表示第几个训练样本， 中括号上标表示第几层，大括号上标表示第几个 mini-batch     



之前我们有    
   $$
   Z^{[l]} = W^{[l]} X + b^{[l]}\\

   A^{[l]} = g^{[l]}(Z^{[l]})

   $$

现在对于每一个mini-batch就会改为

$$
Z^{[l]} = W^{[l]} X^{\{t\}} + b^{[l]}  \\

A^{[l]t} = g^{[l]}(Z^{[l]})
$$

对应的成本函数改为    
$$
J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})

$$
其中：     
1. m是单个minibatch中的样本数
2. $L(\hat{y}^{(i)}, y^{(i)})$ 是一个mini-batch  $X^{\{t\}},Y^{\{t\}}$ 中的样本



如果使用正则化，则

$$
J^{\{t\}} = \frac{1}{l} \sum_{i=1}^{l}  L(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2l} \sum_{l} \lVert w^{[l]} \rVert_F^2,
$$

反向传播也是同理。
我们可以发现对于单个mini-batch，计算步骤和之前普通的batch梯度下降算法是完全相同的。   
**对于每一个mini-batch,都要进行一轮完整的正向和反向传播，计算损失函数，并且更新权重**    

所以使用batch梯度下降算法一次只能梯度下降一次，但是使用mini-batch梯度下降算法每个mini-batch都可以梯度下降一次    
同时，一次遍历所有mini-batch，也就是遍历整个X，称为一个 **迭代（epoch）**



注意：    
1. 样本小于2000个时，不使用mini-batch，    
   样本数量较大时，把一个 mini-batch 的大小设置为64、128、256 或 512 （2的指数）


#### **mini-batch 算法的原理**

在使用batch算法时，成本函数应该是单调递减的，除非学习率过大   
但是对于mini-batch而言，应该如下图,因为每次下降用的都是不同的训练集   

<img src='images/minibatch和batch.png'>    

而更形象地表现如下     
其中:    
蓝线是batch梯度下降算法，也就是把一个mini-batch 的大小设置为 m     
紫线是随机梯度下降算法，也就是把一个mini-batch 的大小设置为 1      
绿线是mini-batch梯度下降算法，也就是把一个mini-batch 的大小设置为 1000   

<img src='images/minibatch的size取值.png'>

这种算法仍然存在如下弊端：    
1. 折线摆动靠近最优解，太慢
2. 学习率不能太大，不然不能在最优解附近收敛


### （二）动量梯度下降算法（Gradient descent with Momentum）

在了解这种算法之前，我们需要先学习一下指数加权平均数

#### （1）指数加权平均数    （Exponentially  weighted averages）·

这是一种便捷地得到动态拟合曲线的方法    

<img src='images/指数加权平均数.png'>    

$
v_0=0 \\
v_1=0.9v_0+0.1\theta_1  \\
v_2=0.9v_1+0.1\theta_2   \\
v_3=0.9v_2+0.1\theta_3  \\
\dots \\
v_t=0.9v_{t-1}+0.1\theta_t  \\
$

其中：   
$v_t$是某一天的加权平均数，$\theta_t$ 是某一天的值     
就可以得到关于 $v_t$ 和 t 的关系图    


公式如下：
$$
v_0=0 \\
v_t=\beta v_{t-1}+(1-\beta)\theta_t  \\
 \\
此时 v_t 可以视为是 \frac{1}{1-\beta}  天的平均值
$$


显而易见, $\beta$ 越趋近于1， 曲线越平缓，对于变化适应的更慢    


<img src='images/指数加权平均数的不同β取值.png'>



由于一开始 $v_0=0$， 所以曲线一开始应该是从0开始的，因此前几天的值拟合的不好    
所以可以选择使用 **指数加权平均的偏差修正 (Bias correction in exponentially weighted averages)**     
公式修改成如下:

$$
v_t=\frac {\beta v_{t-1}+(1-\beta)\theta_t}{1-\beta^t}  \\
$$

注意:     
1. 在实际代码中，一般不把 $v_1, v_2$ 之类变量分开，而是不断地更新v   
    ```python
    v=beta*v+(1-beta)*theta
    ```
2. 偏差修正不是必须的，因为除了开头的几个样本，后面的几乎没有区别。甚至一般不怎么用
3. 动量梯度下降算法其实跟动量没有关系，使用动量Momentum只是一种比喻。      
其实用惯性更好理解。动量大的物体更难以轻易改变运动状态，对应到算法中就是更平滑，更难以产生摆动


#### （2）动量梯度下降算法

无论是batch梯度下降 还是mini-batch梯度下降 ，在每次反向传播的时候都需要计算 $dW^{l},db^{l}$，并且用他们来更新 $W^{l},b^{l}$      
但是这个算法不直接使用 $dW^{l},db^{l}$， 而是使用他们的 指数加权平均数 来更新  $W^{l},b^{l}$     
也就是：      
$$
v_{dW}=\beta v_{dW}+(1-\beta)dW \\
v_{db}=\beta v_{db}+(1-\beta)db \\
W=W-\alpha v_{dW} , b=b-\alpha v_{db}

$$


原理解释     

我们在之前就讲到了minibatch的缺点在于摆动     
取加权平均数可以可以有效减小摆动

注意：   
1. $\beta$ 一般默认设置为0.9   
2. $v_{dW}$ 初始设置应为 和 $dW$ 形状相同的零矩阵，$v_{db}$ 初始设置应为 和 $db$ 形状相同的零矩阵

### (三) RMSprop 算法（root mean square prop算法）

RMSprop 算法 和 动量梯度下降算法Momentum 非常相似，都可以减小摆动，加快逼近速度，允许使用更大的学习率        
只是计算方法有一点点不一样，具体如下：         

$$
S_{dW}=\beta S_{dW}+(1-\beta)dW^2 \\
S_{db}=\beta S_{db}+(1-\beta)db^2 \\ 
\\
W = W-\alpha \frac {dW}{\sqrt{S_{dW}}+10^{-8}} , b = b-\alpha \frac {db}{\sqrt{S_{db}}+10^{-8}}

$$



注意：
1. 这个平方的操作是针对这一整个符号的
2. $+10^{-8}$ 是为了防止分母趋近于0

### （四）Adam优化算法

Adam优化算法就是将Momentum和RMSprop结合在一起     

具体算法如下：
在第 t 次迭代中

$$

v_{dW}=0  \\
S_{dW}=0  \\
v_{db}=0  \\
S_{db}=0  \\

-------------------\\

v_{dW}={\beta}_1 v_{dW}+(1-{\beta}_1)dW \\
v_{db}={\beta}_1 v_{db}+(1-{\beta}_1)db \\

S_{dW}={\beta}_2 S_{dW}+(1-{\beta}_2)dW^2 \\
S_{db}={\beta}_2 S_{db}+(1-{\beta}_2)db^2 \\ 

-------------------\\
\\
v_{dW}^{corrected}=v_{dW}/(1-{\beta}_1^t)    \\
v_{db}^{corrected}=v_{db}/(1-{\beta}_1^t)    \\
S_{dW}^{corrected}=S_{dW}/(1-{\beta}_2^t)    \\
S_{db}^{corrected}=S_{db}/(1-{\beta}_2^t)    \\

-------------------\\
W = W-\alpha \frac {v_{dW}^{corrected}}{\sqrt{S_{dW}^{corrected}}+10^{-8}} \\
b = b-\alpha \frac {v_{db}^{corrected}}{\sqrt{S_{db}^{corrected}}+10^{-8}}

$$

注意：     
1. 其中 ${\beta}_1$ 建议设置为0.9，${\beta}_2$ 建议设置为 0.999

关于Momentum,RMSprop和Adam 算法能够加快收敛速度的解释：   

首先对于比较大的神经网络而言，由于W参数很多，所以不太可能会困在局部最优解，因为这要求很多参数同时为凹函数或者凸函数

<img src='images/局部最优.png'>


真正影响收敛速度的是下面这种鞍形。在鞍中点的斜率也是0

<img src='images/马鞍图.png'>

如果鞍的中间区段比较缓的话，这段时间的收敛就会很慢      
而使用这些算法，由于‘动量’很大，所以可以快速地度过这些区域

### （五）学习率衰减（Learning Rate Decay）

由于学习率 $\alpha $ 是固定值，所以最后可能无法完全收敛到最优解，而是在最优解附近摆动。     
这就需要我们让学习率不断降低，来收敛到最优解      
所以对a的修改如下：   

$$
a=\frac{1}{1+decayrate*epoch\_num} a_0
$$

其中：     
$a_0$为初始学习率     
epoch_num 是当前是第几个 epoch     
decayrate 是衰减率，也是一个超参数

# 第六部分：超参数调试，Batch正则化，和程序框架

### (一)如何选择最合适的超参数

我们已经知道很多的超参数了，但是这些超参数的重要性是不同的。    
优先级大到小排列如下:     
1. 学习率 $ \alpha $   
2. 动量梯度下降算法的 $ \beta $, 隐藏层单元，mini-batch size   
3. 层数，学习率衰减率  >  Adam 算法的 $ \beta_1,\beta_2,\epsilon$

那如何测试出最好的超参数呢?


对于一组超参数而言，如下图所示，当我们测试的次数有限时，建议采用右边的取点方法。     

<img src='images/超参数组合.png'>

因为左图中每个参数只采用了5个值，而右图中每个参数尝试了25个值，所以可以更好地观察哪个参数更重要    
拓展到3个参数的组合尝试中也是如此

在所有的采样点中测试出最好的之后，可以选择进一步精细化测试    

<img src='images/更精细化.png'>

此外，对于单个参数在一定范围内的取点测试，不建议均匀分布随机取点      
举个例子，如果某个参数要从50试到100，那么我们可以均匀取样     
但是如果某个参数要从0.0001 试到 1， 那么最好像 0.0001， 0.001 ，0.01， 0.1， 1  这样取样（但仍需是随机取样）      
在python中可以这样实现:     
```python
r=-4* np.random.rand()  # r 属于[-4,0]
a = 10**r               # a 属于[10-4,1]

```

那么对于像 $\beta$ 这样的0.9 到0.999之间取样的该如何实现呢？     
```python
r=-2* np.random.rand()-1  # r 属于[-3,-1]
beta = 1-10**r            # beta 属于[0.999,0.9]

```


为什么要这样取点呢？因为在深度学习中，很多时候在某个范围内参数的影响力会突然变大很多       
比如在 $\beta$ 靠近1时，所代表的平均天数 1/(1-$\beta$)  会变大得非常迅速，所以要在这个范围内尝试更多的点

然而，我们上面讨论的都是给定参数后，完整地测试一次之后对各组的测试结果进行比较     
但是如果进行一次实验的成本非常高，时间非常长，我们只能做一两次实验，该怎么确定最优参数呢？

如下图，我们可以采用这种 **Babysitting one Model**      
也就是在指定的时间间隔内调整一次参数，观察输出的成本函数的变化     


<img src='images/Babysitting one model.png'>

### (二) Batch归一化 （Batch Normalizing）

我们之前讲过归一化输入来把原始数据X 的均值降到0， 方差缩放到1·：     

$$
X_{\text{std}} = \frac{X - \mu}{\sigma}
$$



其中 $\mu$ 是特征的均值，$\sigma $ 是特征的标准差。  
$$
\mu = \frac{1}{m} \sum_{i=1}^m x_i
$$
$$
\sigma = \sqrt{\frac{1}{m} \sum_{i=1}^m (x_i - \mu)^2}
$$

我们知道这种方法把扁平的数据拉伸得饱满，可以加快学习过程

那么我们不经想到，既然可以把输入样本 X 进行归一化，是不是可以把每一层的输出 a 都归一化呢？    
事实上这是可以的，但是我们一般归一化的是 $z^{[l]}$ 而不是  $a^{[l]}$， 归一化可以使下一层的 w 和 b 训练得更快

假设第l层有m个隐藏节点      





其中 $\mu$ 是特征的均值，$\sigma $ 是特征的标准差。  
$$
\mu = \frac{1}{m} \sum_{i=1}^m Z^{[l](i)}
$$
$$
\sigma^2 = \frac{1}{m} \sum_{i=1}^m (Z^{[l](i)} - \mu)^2
$$


$$
Z_{norm}^{[l](i)} = \frac{Z^{[l](i)} - \mu}{\sqrt{\sigma^2+\epsilon}}
$$


注： $\epsilon$ 用于防止分母为0

现在的每个隐藏单元都含有平均值0和方差1，但是也许隐藏单元有了不同的分布会有意义，所以我们手动修改其平均值和方差     
$$
\tilde{Z}^{[l](i)} = \gamma*Z_{norm}^{[l](i)} + \beta,
$$

这样我们就可以随意修改 $ Z^{[l](i)} $ 的平均值     

在神经网络中，我们不再使用  $ A^{[l]}= g^{[l]}(Z^{[l](i)})$, 而是 $ A^{[l]}= g^{[l]}(\tilde{Z}^{[l](i)})$



注意:       
1. 在过程中我们引入了新的参数 $ \gamma^{[l]} ， \beta^{[l]}$，这对于每层是不一样的
2. 这里的 $\beta$ 和 指数加权平均数（比如Adam算法等）中的 $\beta$ 不是一个东西，这里的是用于正则化控制平均值的
3. $ \gamma^{[l]} ， \beta^{[l]}$ 并不是超参数！这是需要更新的参数，和W跟b一样！    
   我们可以使用任意的优化算法，可以是Adam、RMSprop、Momemtum, 或者只是普通的梯度下降    
   然后更新参数,例如：    
   $$
    \beta^{[l]} = \beta^{[l]} - \alpha* d\beta^{[l]}
   $$
4. 实践中，Batch归一化通常和 mini-batch一起使用,每一个mini-batch $ X^{{1}}$ 中的计算方法都和上面相同    
5. 事实上，由于在Batch归一化的过程中首先把平均值拉到了0，所以实际上 $b^{[l]}$ 被约掉了。可以暂时把它设置为0
6. $Z^{[l]}$ 的维数是 $(n^{[l]},1)$        
   $b^{[l]}$ 的维数是 $(n^{[l]},1)$   
   $\beta^{[l]}$ 的维数是 $(n^{[l]},1)$    
   $\gamma^{[l]}$ 的维数是 $(n^{[l]},1)$     
   



**Batch 归一化的原理：**     
除了我们之前说过的，归一化可以把扁平的数据变得饱满从而加速计算之外，Batch归一化还有其他优点：     
Batch归一化通过稳定每层的输入，使得深层网络的训练变得更加稳健。权重更新不再依赖于前一层特定的权重分布，      
因此，每一层都可以在相对独立的环境中学习和适应，这减少了所谓的“内部协变量偏移”（即训练过程中层激活分布的变化）。

通过确保每层的输入保持相同的分布，它减少了层与层之间参数更新的相互依赖性。       
这意味着即使前面层的参数发生了变化，也不会极大地影响到后面层的学习进程。      
这种特性使得每一层都能在更加独立的环境中进行学习，降低了层间复杂的相互作用，有助于网络更快地收敛。

即使训练数据在某些方面不够代表性（如只有黑猫），Batch归一化也有助于模型在面对不同类型（如有色猫）的数据时，保持稳定的性能。     
因为Batch归一化减少了模型对输入数据具体特征的敏感度，使得模型对于输入数据中的细微变化更加鲁棒。


**Batch 归一化的副作用**     
这种方法有轻微的正则化效果。     
原因是在每个mini-batch中计算均值和方差，而不是在整个数据集上，用一小部分的数据计算导致了每个隐藏层上都有噪音     
功能和dropout类似，迫使后部单元不过分依赖任何一个隐藏单元，但是效果比较轻微      
因此单个 mini-bacth 越大，Batch归一化的正则化效果越弱

我们现在已经知道了如何在训练过程中使用Batch归一化，但是放到测试时，该怎么做呢？        
我们现在对于单个mini-batch中的某个隐藏层的计算方法如下，其中 m 是这个mini-batch中的样本数量，而不是整个训练集

$$
\mu = \frac{1}{m} \sum_{i=1}^m Z^{[l](i)}   \\

\sigma^2 = \frac{1}{m} \sum_{i=1}^m (Z^{[l](i)} - \mu)^2   \\

Z_{norm}^{[l](i)} = \frac{Z^{[l](i)} - \mu}{\sqrt{\sigma^2+\epsilon}}   \\

\tilde{Z}^{[l](i)} = \gamma*Z_{norm}^{[l](i)} + \beta,
$$


但是问题是，在测试时，我们是逐个测试样本的，而不是把一个mini-batch中的所有样本同时处理      
而单个样本的 $均值\mu, 方差\sigma^2$  没有意义，因此需要单独估算 $\mu, \sigma^2$， 这可以使用指数加权平均数来实现      


对不同的mini-batch $X^{\{1\}},X^{\{2\}},X^{\{3\}},……$ 的第l层训练时，我们得到了$\mu^{\{1\}[l]},\mu^{\{2\}[l]},\mu^{\{3\}[l]},……$      
然后对这些值进行指数加权平均就可以得到最后测试集所需要的 $\mu$ 了， $\sigma^2$ 也是同理      

最后在测试时，也要进行对应的batch归一化，用的就是测试集的 Z 和 用指数加权平均数 估算出来的  $\mu$ 和  $\sigma^2$

### （三） Softmax回归

我们之前学的都是二分类，但是如果要实现多分类，该如何实现呢？


我们可以使用Softmax回归层，使得最后输出的结果是一个数组。    
假设我们要识别猫、狗和鸡， 那么这个输出数组可以是 [ 0.1 , 0.5 , 0.4 ] ,这样显而易见狗的概率最大。      
同时要确保数组中的所有概率之和为 1        
之所以叫Softmax， 是因为形如 [0, 1, 0] 这样的只有一个1的全零数组叫Hardmax


我们用 C 来表示类别个数，Softmax层的计算方法如下：     

$$

z_i^{[1]} = w_i^{[1]T} x + b_i^{[1]}  

$$
$$

a_i^{[l]}=\frac{e^{z_i^{[1]}}}{\sum_{j=1}^C e^{z_j^{[1]}}}
$$

注意:    
Z应该是一个4*1的数组


我们可以把上面的第二步当做Softmax的激活函数     

$A^{[l]}=g^{[l]}(Z^{[l]})$

这一激活函数的特殊之处在于，这个激活函数 𝑔 需要输入一个 4×1维向量，然后输出一个 4×1维向量      
而之前，我们的激活函数都是接受单行数值输入，例如 Sigmoid和 ReLu激活函数，输入一个 实数，输出一个实数。

在图像上，可以这么理解      

<img src='images/Softmax.png'>

我们可以发现，任何两个分类之间的决策边界都是线性的

注意：    
1. Softmax通常被用作神经网络的最后一层。    
它的作用是将网络的原始输出，即logits转换成概率分布，这有助于直接从输出层解读每个类别的预测概率
2. 如何知道输出数组中的每个值代表哪个类别？      
在训练神经网络时，你需要预先定义类别的顺序，并在整个模型训练和预测过程中保持这一顺序不变。

**Softmax的损失函数**       
由于现在最后的输出不是单个值，而是一个数组，所以损失函数也要重新定义     
在Softmax分类中，一般用的损失函数是：
$$

L(\hat{y}, y) = -\sum_{j=1}^{4} y_j \log \hat{y}_j

$$

举个例子：    
$$
y=[0,   1,   0,    0]  \\
\hat{y} = [0.3,  0.2,  0.1,  0.4]
$$

在这种情况下，想要让损失函数尽可能小，就必须让 $\hat{y}_2$ 尽可能大，也就是尽可能准确






上面是单个样本的损失函数，整个训练集的成本函数定义如下：     
$$
J(w^{[1]}, b^{[1]}, \ldots) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{(i)}, y^{(i)})

$$



关于向量化时的数据尺寸如下    

<img src='images/Softmax中的Y和Y_hat.png'>

如果C=4，那么 $y^{(1)},y^{(2)},…… , y^{(m)} $ 都是 4 × 1 的向量， 而Y最终是一个 4 * m 的矩阵，      
同样的 $ \hat{Y}$ 也是一个 4 * m 的矩阵

最后我们来看一下，在有Softmax输出层时如何实现梯度下降法:      
输出层的尺寸是 C * 1     
计算方法如下：     
$$
dZ^{[l]} = \hat{y}-y
$$


现在看起来Softmax可能有点抽象，但是我们会在下面的卷积神经网络中用具体的图像识别例子来理解

# 第七部分：如何优化机器学习项目

在这一部分中，技术性的东西会比较少，更多的是一些项目优化的经验和策略

#### （一）正交化（orthogonalization）

搭建建立机器学习系统的挑战之一是，有那么多的超参数可以调， 你可以尝试和改变的东西太多太多了。     
而所谓 **正交化** , 就是每一个调整的内容只影响某一个性质，否则调整一个内容但是影响了很多性质，会让调试变得困难。


我们在前面说过样本要分为训练集，开发集和测试集。    
我们首先要保证在训练集上效果不错，其次是开发集，然后是测试集，最后是实际应用中。   
在不同的阶段出现问题对应的解决办法是不一样的。   

<img src='images/chain of assumptions in ML.png'>

1. 在训练集上表现不好：  使用更大的网络，使用更好的算法（比如Adam）
2. 在训练集上表现好，但是在开发集上表现不好： 正则化，增大训练集
3. 在开发集上表现好，但是在测试集上表现不好： 更大的开发集，
4. 在测试集上表现好，但是在实际中表现不好： 改变开发集或者成本函数

#### （二）单一数字评估指标

如果你有一个单实数评估指标，你的进展会快得多     
但是有些时候并没有那么容易确定 单实数评估指标，请看下例   

**示例一**    
查准率（Precision）是在你的分类器标记为猫的例子中，有多少真的是猫。      
查全率（recall）是对于所有真猫的图片，你的分类器正确识别出了多少百分比     

<img src='images/查准率和查全率.png' width=700>

那么这两个指标该选哪一个作为评估指标呢？   
一般建议取调和平均数 $F_1$ 进行比较：     
$$
F_1=\frac{2}{\frac{1}{P}+\frac{1}{R}}
$$

$F_1$ 越大，代表准确率越高

**示例二**     
看下面这个例子，准确率和运算时间之间，应该如何权衡呢？

<img src='images/满足和优化指标.png'>


事实上，比如在1000ms内用户感觉不出区别，所以只要满足运算时间小于1000ms即可，只需要在剩下的里面比较准确率，所以选B      
更一般地说，如果要考虑N个指标，有时候可以只选择其中一个作为评判标准，剩下的只要满足某个要求就可以了

你已经学过如何设置开发集和评估指标，就像是把目标定在某个位置，让你的团队瞄准。    
但有时候在项目进行途中，你可能意识到，目标的位置放错了。这种情况下，你应该移动你的目标。       
如果你对旧的错误率指标不满意，就不要一直沿用你不满意的错误率指标，而应该尝试定义一个新的指标，能够更加符合你的偏好，定义出实际更适合的算法。


#### （三）训练 /开发 /测试集划分

机器学习中的工作流程是，你尝试很多思路，用训练集训练不同的模型，然后使用开发集来评估不同的思路，      
然后选择一个，然后不断迭代去改善开发集的性能，直到最后你可以得到一个令你满意的成本，然后你再用测试集去评估。

同时，开发集和测试集需要来自相同的分布，不可以开发集的图片都是北半球，测试集的图片里都是南半球的

关于大小的划分，我们之前说过是 7:3 或者 6:2:2 , 但是如果数据体量上到百万级别， 98:1:1 会更合理。毕竟测试集只要反映全面性能就可以了。

#### （四）人的表现 Human-level performance

在不断地提升准确率的同时，AI的能力会超过人类    
但是如下图所示，在能力超过人类后，进步速度会变慢，并且最后无论如何也不能超过一个所谓的“**贝叶斯最优错误率（ Bayes optimal error）**”    
贝叶斯最优错误率一般认为是理论上可能达到的最优错误率

<img src='images/Human-level performance.png'>

贝叶斯错误率或者对贝叶斯错误率的估计 和 训练错误率 之间的差值称为 **可避免偏差**，    
理论上是不可能超过贝叶斯错误率的，除非过拟合

注意: 人类水平 是指人类可以达到的最高水平 而不是平均水平！ 而贝叶斯最优错误率必须要比人类水平 高一点或很多！     

<img src='images/Reducing bias and variance.png'>

#### （四）迁移学习

也就是在别人训练好的模型和参数上继续训练

举个例子，假如说你要建立一个猫咪检测器，用来检测你自己的宠物猫。      
假如你的两只猫叫 Tigger和Misty，所以要做一个三分类问题，图片里是 Tigger还是 Misty，或者都不是，（忽略两只猫同时出现在一张图片里的情况）     
现在你可能没有Tigger或者 Misty的大量的图片，所以你的训练集会很小，你该怎么办呢？   

可以从网上下载一些神经网络开源的实现，不仅把代码下载下来，也把权重下载下来。     
举个例子， 假设这个网络使用ImageNet数据集进行训练，那么这个网络最后的 Softmax层 会进行1000分类。  

你可以去掉这个Softmax层，创建你自己的 Softmax单元，用来输出 Tigger、 Misty和neither三个类别。          
就网络而言，最好冻结网络中所有层的参数，只需要训练和你的 Softmax层有关的参数。

如何实现冻结呢？    
大多数深度学习框架都支持这种操作，取决于用的框架，它也许会有trainableParameter=0 这样的参数，有时也会有 freeze=1 这样的参数。


上面说的是 手里有的数据集很小 的情况，如果你有一个更大的标定 的数据集（有大量的 Tigger和 Misty的照片），      
你应该冻结更少的层，然后训练后面的层     
如果你有越来越多的数据，你需要冻结的层数越少，你能够训练的层数就越多      
极端情况下 ，你可以用下载的权重只作为初始化，用它们来代替随机初始化，接着你可以用梯度下降训练，更新网络所有层的所有权重。

# 第八部分：卷积神经网络

在计算机视觉中有一个问题，就是输入的数据可能会很大。   
我们之前讲到的神经网络连接方式都叫做 **“全连接”**，而我们之前用的都是64 * 64 * 3 的小图片，所以还没什么问题     
但是当图片尺寸比较大时，就很难处理，比如 1000 * 1000 * 3      
因此我们需要使用 **卷积计算**


### （一）卷积计算

计算机视觉最底层的原理是边缘检测， 这个我们在之前就已经讲到过     
前面几层检测边缘，中间的层检测部分区域，后面的层检测完整物体        

<img src='images/边缘检测.png'>   

更细化一点，要分为垂直边缘和水平边缘     

<img src='images/垂直边缘和水平边缘.png' width=500>

检测出这些边缘就需要用到 **卷积运算**


我们以垂直边缘检测为例，假设有一张 6 * 6 * 1 的灰度图像     
我们需要像下面这样做卷积计算     

<img src='images/Vertical edge detection.png'>

注意:     
1. 中间的星号是数学中的卷积运算符，而不是python中的元素乘法。在python中式conv_forward( )，在Tensorflow中是 tf.conv2d
2. 当前构造了一个3 * 3 的矩阵，被称为过滤器， 也有时被称为 核
3. 在数学中进行卷积计算时，一般先把过滤器进行镜像对称，但是在神经网络中我们不这么做（其实我们现在的计算方法在数学中叫做互相关）

为什么这个可以做垂直边缘检测呢？让我们来看另外一个例子。      

<img src='images/垂直边缘检测原理.png' width=500>     
<br>
<img src='images/暗边缘检测.png' width=500>

最后的图中有一个特别明显的垂直边缘线，目前之所以有点太宽了，是因为图片太小了。图片大了就正常了     
如果不在乎中间是明线还是暗线，可以取绝对值

我们刚刚看了垂直边缘检测的过滤器，同理也可以知道水平边缘检测的过滤器     

<img src='images/Verticle and Horizontal Edge detection.png'>

再看一个复杂一点的例子：     

<img src='images/复杂一点的边缘检测.png'>   

由于我们的图片尺寸太小，所以会有10和-10的过渡带，图片大一点就不会了

除了使用 1,0,-1 之外，其实还有一些数字选择       
$$
\text{Sobel filter matrix:}
\begin{bmatrix}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix}
$$
$$
\text{Scharr filter matrix:}
\begin{bmatrix}
3 & 0 & -3 \\
10 & 0 & -10 \\
3 & 0 & -3
\end{bmatrix}
$$

这些筛选器有时可以提升鲁棒性，     
现在甚至可以把这9个数字设置为9个参数，并使用反向传播算法来更新他们      
这种过滤器对于数据的捕捉能力甚至可以胜过任何之前这些手写 的过滤器。     
相比这种单纯的垂直边缘和水平边缘，它可以检测出45°或 70°，甚至是任何角度的边缘    
我们会在迟一点学到这种过滤器

### （二）Padding

假设图像尺寸 n * n , 过滤器尺寸 f * f， 那么输出的维度就是 (n- f +1) * (n- f +1)

这里有2 个问题：    
1. 每次做卷积原图片都会变小
2. 图像中间的像素点被卷积计算了好几次，但是图像边缘的像素点没有被计算那么多次 。所以可能导致边缘的特征没有被捕捉到    

因此，解决办法是在卷积操作前填充图像，比如在 6 * 6 的外边包裹一层像素， 变成 8 * 8     
再用 3 * 3 的过滤器来卷积，就仍然得到 6 * 6      

习惯上，可以用0来填充     
如果填充的层数是 p , 输出就变成了 (n- f +2p +1) * (n- f +2p +1)    

刚刚我们提到的是 p=1 的情况，至于填充多少像素，通常有两个选择：     
1. Valid卷积： p=0， 不填充
2. Same卷积： p=(f-1)/2  卷积后图片大小不变  (一般把 f 设置为奇数)

### （三）卷积步长（Strided Convolutions）


目前我们都是一次移动1格， 但是如果把 步长 s 设置为2， 过滤器一次会跳过两个格子     


此时输出尺寸变为：$ \lfloor (\frac{n+2p-f}{s}+1) \rfloor \times \lfloor (\frac{n+2p-f}{s}+1) \rfloor$     

其中 $\lfloor$ $\rfloor $ 是向下取整,     
这意味着只在蓝框完全包括在图像或填充完的图像内部时，才对它进行运算。如果有任意一个蓝框移动到了外面，那就不要进行相乘操作。

### （四）三维卷积

我们刚刚讲的都是灰度图像，那么对于RGB图像 比如 6 * 6 * 3， 该如何卷积呢？    

你可以把它想象成 3 张 6 * 6 的图像的堆叠。现在我们不是把它和原来的 3 * 3 的过滤器做卷积，而是一个 3 * 3 * 3 的三维过滤器     

<img src='images/Convolution on RGB image.png' width=500>    

  

现在第一个6代表图像宽度，第二个6代表图像宽度，3 代表通道数目。图像的通道数必须和过滤器的通道数匹配

如果指向检测红色通道的边缘，可以将第一个过滤器设置为：    
\begin{bmatrix}
1 & 0 & -1 \\
1 & 0 & -1 \\
1 & 0 & -1
\end{bmatrix}
剩下的绿色通道和蓝色通道的过滤器都设置为0     
那么这个就是一个检测垂直边界的过滤器，但是只对红色通道有用     
或者如果你不关心垂直边界在哪个颜色通道里，那就把 绿色通道和蓝色通道的过滤器 都设置成和 红色通道 一样就好了

如果不仅仅想要检测垂直边缘，而是要同时检测垂直边缘和水平边缘，甚至还有 45°倾斜的边缘，那么可以使用多个过滤器    

<img src='images/Multiple filters.png' width=600>   

把两个输出按顺序堆叠，就得到了一个 4 * 4 * 2 的输出立方体， 这里的2 来自于使用了几个不同的过滤器

总结一下， 如果输入图像为 $ n \times n \times n_c$ ，其中 $n_c$ 是通道数目   
然后卷积上 $n_{c^{'}}$ 个 $f \times f \times n_c$ 的过滤器，最后得到的输出就是：

$ \lfloor (\frac{n+2p-f}{s}+1) \rfloor \times \lfloor (\frac{n+2p-f}{s}+1) \rfloor  \times n_{c^{'}}$      

$n_{c^{'}}$ 是过滤器的个数，也是下一层的通道数

### (五)池化层 （Pooling Layers）

除了卷积层之外，池化层也可以减小图片的大小，提高计算速度，同时提高所提取特征的鲁棒性

我们一般采用的池化方法是 **最大池化(Max pooling)** ， 也就是取一个特定区域内的最大值

在下图中我们使用了 2 * 2 区域， 步长为 2 。 即 f=2 ， s =2

<img src='images/Max pooling.png'>

之前讲的计算卷积层输出大小的公式同样适用于最大池化，即： 
$ \lfloor (\frac{n+2p-f}{s}+1) \rfloor \times \lfloor (\frac{n+2p-f}{s}+1) \rfloor \times n_c $           

如果输入是三维的，输出也是三维的，通道数不变，宽高公式如上



池化层的原理：    
数字大意味着可能探测到了某些特定的特征。     
最大化操作的功能就是只要在任何一个象限内提取到某个特征，它都会保留在最大化的池化输出里。     
所以最大化运算的实际作用就是，如果在过滤器中提取到某个特征，那么保留其最大值。

注意：    
1. 池化层有一组超参数，但并没有参数需要学习。一旦确定了 𝑓和 𝑠，它就是一个固定运算，梯度下降无需改变任何值。     
因此有时候池化层不被作为一个层，或者和一个卷积层一起作为一个层
2. 还有一种池化叫平均池化，不太常用
3. 大部分情况下，最大池化很少用padding

### （六）卷积神经网络

我们先来看单层卷积神经网络     

<img src='images/卷积神经网络的一层.png' width=600>    


我们之前讲过：    
   $$
   Z^{[l]} = W^{[l]} X + b^{[l]}\\

   A^{[l]} = g^{[l]}(Z^{[l]})

   $$

这里的过滤器都可以用 $ W^{[l]}$ 来表示，激活函数可以使用 Relu   
记得在激活函数前还有加上偏差b，用python广播机制实现     
注意，无论输入的图片有多大，需要更新的参数都是取决于过滤器，即 $n_{c^{'}} \times f \times f \times n_c$    
$n_{c^{'}} $ 就是过滤器的数量，也是提取的特征数     


总结一下尺寸:  


输入尺寸：$n_H^{[l-1]} \times n_W^{[l-1]} \times n_c^{[l-1]} $    

输出尺寸：$n_H^{[l]} \times n_W^{[l]} \times n_c^{[l]}$    

筛选器尺寸：$f^{[l]} \times f^{[l]} \times  n_c^{[l-1]} $     

Weights(多个筛选器)： $f^{[l]} \times f^{[l]} \times  n_c^{[l-1]}  \times n_c^{[l]} $    

偏差bias： $ 1 \times 1 \times 1 \times n_c^{[l]}$

$A^{[l]}$ : $ m \times n_H^{[l]} \times n_W^{[l]} \times n_c^{[l]}$    


其中 ：    
$$ n_H^{[l]} = \lfloor \frac{n_H^{[l-1]}+2p^{[l]}-f^{[1]}}{s^{[l]}}+1 \rfloor $$
$$ n_W^{[l]} = \lfloor \frac{n_W^{[l-1]}+2p^{[l]}-f^{[1]}}{s^{[l]}}+1 \rfloor $$

现在我们已经知道了如何为卷积神经网络构建一个卷积层，那如何构建一个深度卷积神经网络呢？     

如下图所示,假设要识别输入的 39 * 39 * 3 的图像里有没有猫，每一层都使用了 valid卷积，也就是padding为0      
第一层用了 10 个过滤器，第二层用了 20个过滤器，第三层用了 40 个过滤器      
具体参数见图中

<img src='images/Example Convnet.png' width=600>    


到此这张39 * 39 * 3 的图像处理完毕了，最后提取了 7 * 7 * 40 个特征     
然后对该卷积进行处理，可以将其平滑或展开成 1960个单元，输出为一个向量，      
其填充内容是logistic回归单元还是 softmax回归单元取决于我们是想识图片上有没有猫，还是想识别 𝐾 种不同类型中的一个   

举个例子：假设要分为10种类型，那应该怎么处理最后一个卷积层 7 * 7 * 40 的输出？     


1.  将最后一个卷积层的输出，即 7 * 7 * 40 的三维特征图（feature map），**展开** 成一个一维的长为1960的向量。
2.  在Softmax层， 将 1960 * 1 的矩阵 使用一个全连接层 转化成 10 * 1 的向量。 W的尺寸是 10 * 1960，  
$$
z_i^{[1]} = w_i^{[1]T} x + b_i^{[1]}  
$$

3.  在Softmax层，对于每个类别 k，你将计算该类别的得分的指数与所有类别得分的指数和的比例。公式如下：
$$
a_i^{[l]}=\frac{e^{z_i^{[1]}}}{\sum_{j=1}^{10} e^{z_j^{[1]}}}
$$
  


整个神经网络代码如下:（GPT给出，存在问题，仅供理解）


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(10, (3, 3), activation='relu', input_shape=(39, 39, 3), padding='valid'),
    tf.keras.layers.Conv2D(20, (3, 3), activation='relu', padding='valid'),
    tf.keras.layers.Conv2D(40, (3, 3), activation='relu', padding='valid'),
    tf.keras.layers.Flatten(),  # Flatten the output of the last conv layer
    tf.keras.layers.Dense(10, activation='softmax')  # Dense layer with softmax activation for 10 classes
])


# 这个示例建立了一个简单的卷积神经网络，其中包括三个卷积层和一个输出层，输出层用于10类的分类任务。
# 展开操作是通过 `Flatten` 层自动处理的，而分类是通过具有Softmax激活的 `Dense` 层实现的。

注意:    
1. 随着神经网络加深。图片宽高应该变小，通道数应该变大
2. 一个典型的神经网络通常有 3 层， 。
3. 卷积层用CONV 来表示， 池化层用 POOL 来表示， 全连接层用FC 表示

现在我们要往卷积神经网络中加入池化层，该如何实现呢？     

下面给出了一个例子，用于识别手写数字是 1 - 9，参考的参数来源是LeNet-5模型，但这里并不是LeNet-5模型

<img src='images/LeNET-5.png' width =650> 

关于这个模型，我们有几点需要注意：     
1. 第一层的过滤器 f =5, stride=1, 使用了6个过滤器，padding=0， 使用了ReLU函数，其它层类似如图
2. 由于池化层没有需要更新的参数，所以把一个池化层和一个卷积层合起来作为一个层
3. 全连接层 FC3 就是普通的神经网络，权重W尺寸为 120 * 400
4. 随着神经网络的深度不断加深，高度 $n_H$ 和 宽度 $n_W$ 通常会减少，通道数量会增加
5. 在神经网络中，另一种常见模式就是一个或多个卷积后面跟随一个池化层，然后再是一个或多个卷积层后面再跟一个池化层，然后是几个全连接层，最后是一个
softmax。
6. 参数的大小如下图，可以看见大部分参数在全连接层：

<img src='images/卷积神经网络中的参数表.png' width=500>    




### （七）经典的神经网络

#### (1) LeNet-5 网络

LeNet-5是 针对灰度图片训练的，所以图片的大小只有 32×32×1, 用于识别手写数字     

<img src='images/Letnet网络.png' width = 600>

注意：    
1. 在这篇论文写成的那个年代，人们更喜欢使用平均池化，而现在我们可能用最大池化更多一些。
2. 这个神经网络中还有一种模式至今仍然经常用到，就是一个或多个卷积层后面跟着一个池化层，然后又是若干个卷积层再接一个池化层，然后是全连接层，最后是Softmax和输出

#### （2） AlexNet 网络

AlexNet首先用一张 227×227×3的图片作为输入，实际上原文中使用的图像是 224×224×3
但是如果你尝试去推导一下，你会发现 227×227这个尺寸更好一些。


<img src='images/AlexNet.png' width =600>


注意：     
1. 实际上，这种神经网络与LeNet有很多相似之处，不过 AlexNet要大得多。 LeNet-5大约有 6万个参数，而 AlexNet包含约 6000万个参数。
2. AlexNet比 LeNet表现更为出色的另一个原因是它使用了 ReLu激活函数。
3. 第一点，在写这篇论文的时候， GPU的处理速度还比较慢，所以 AlexNet采用了非常复杂的方法在两个 GPU上进行训练。大致原理是，这些层分别拆分到两个不同的GPU上，同时还专门有一个方法用于两个 GPU进行交流。
4. 论文还提到，经典的AlexNet结构还有另 一种类型的层，叫作 “局部响应归一化层”，即 LRN层，这类层应用得并不多

#### （三）VGG-16

VGG-16网络没有那么多超参数，这是一种只需要专注于构建卷积层的简单网络。

<img src='images/VGG-16.png' width=600>

注意：   
1. 这里采用的都是大小为 3×3 步幅为 1的过滤器，并且都是采用 same卷积
2. VGG-16的这个数字 16，就是指在这个网络中包含 16个卷积层和全连接层。共包含约 1.38亿个参数，即便以现在的标准来看都算是非常大

### （八） 1×1 卷积

当卷积层的过滤器尺寸为1 * 1 时，另有妙用

如果某一层的图片输出已经是 6 * 6 * 32 ，对他进行 1 * 1 卷积所实现的功能就是遍历这 36 个单元格，     
计算每个单元格的32个数字之和，然后使用ReLU激活     

<img src='images/1×1卷积.png'>

这种方法通常 称为 1×1卷积，有时也被称为 Network in Network     
这个方法可以在不改变宽高的情况下，用于压缩通道数 或者 自由地控制通道数        
而相反池化层则是压缩宽高，不改变通道数

# 第九部分：残差网络（ResNetS）

非常非常深的神经网络是很难训练的，因为存在梯度消失和梯度爆炸问题。     
而跳跃连接（ Skip connection），它可以从某一层网络层获取激活，然后迅速反馈给另外一层，甚至是神经网络的更深层。    
我们可以利用 跳跃连接 构建残差网络 ResNets， 有时深度能够超过 100层

ResNets网络是由残差块（Residual block）构成的, 甚至是残差块呢？    


普通的神经网络的计算步骤是这样的


   $$
   Z^{[l+1]} = W^{[l+1]} A^{[l]} + b^{[l+1]}\\

   A^{[l+1]} = g^{[l+1]}(Z^{[l+1]})  \\


   Z^{[l+2]} = W^{[l+2]} A^{[l+1]} + b^{[l+2]}\\

   A^{[l+2]} = g^{[l+2]}(Z^{[l+2]})
   $$


而残差块会修改如下:     


<img src='images/Residual Block.png' width =600>

也就是把最后一步改成了    
$$
   A^{[l+2]} = g^{[l+2]}(Z^{[l+2]}+A^{[l]})
$$
也就是把$A^{[l]}$ 直接拷贝到神经网络的深层      
上面的这一整个叫一个残差块，这里选择跳跃连接了1层，有时可以跳跃连接好几层

一个ResNet网络就是很多这样的残差块堆积在一起，可以形成很深的神经网络    

<img src='images/ResNet.png' width=600>     

把普通网络 (Plain Network) 变成ResNet的方法是加上所有跳跃连接，比如每两层增加一个捷径，构成一个残差块。
如图所示 ，5个残差块连接在一起构成一个残差网络。
残差网络相比普通网络的性能差别如下：   

<img src='images/残差网络和普通网络的比较.png' width =600>

**残差网络的原理**      
残差网络使得网络在增加深度的同时能有效避免梯度消失或爆炸的问题，保持网络训练的稳定性，从而实现更深层次的网络构建而不会导致性能下降。

1. **跳跃连接**：即使更深层的网络层没有学到有效的特征，网络仍然可以利用之前层的特征学习。
2. **梯度流**：在反向传播时，跳跃连接提供了一条没有任何非线性变换的直接路径，这意味着梯度可以更直接地流回输入层，避免了在深层网络中常见的梯度消失问题。因此，即使网络非常深，梯度也能有效地传播



# 第十部分：谷歌 Inception 网络

构建卷积层时，你要决定过滤器的大小究竟是1×1、3×3 或是 5×5，或者要不要添加池化层。      
而 Inception网络的作用就是代替你来决定，虽然网络架构因此变得更加复杂，但网络表现却非常好     

单个Inception的原理如下：  
输入为 28×28×192，输出为 28×28×256

<img src='images/Inception网络的原理.png' width=600>

这里使用的都是same卷积

然而我们计算发现，光是 5 * 5 卷积就要进行 （5 * 5 * 192）*（28 * 28 * 32）= 1.2 亿次乘法运算      
这种计算成本太高了，所以需要使用 1 * 1 卷积来压缩通道数

<img src='images/压缩Inception通道数.png' with =600>

因此有时中间这层被称为瓶颈层，这使得前后加起来的总计算次数约为 1204万，几乎下降到了十分之一     
你可能会问，仅仅大幅缩小表示层规模会不会影响神经网络的性能？    
事实证明，只要合理构建瓶颈层，你既可以显著缩小表示层规模，又不会降低网络性能，从而节省了计算。    

同理对其他层也进行优化，最后的单个模块如下  

<img src='images/优化过的Inception模块.png' width=600>

注意：     
关于最后一个池化，我们使用same padding之后，输出将会是 28×28×192， 通道数太多，    
所以在加上一个 1 * 1 的池化层， 使其缩小到 28×28×32

把很多Inception 模块连接起来，就是Inception 网络   

<img src='images/Inception Network.png' width=600>   

我们可以发现，这里其实还有一些分支，这些分支有什么用呢？     
在网络的最后几层，通常称为全连接层，在它之后是一个Softmax层来做分类     
这些分支所做的就是通过隐藏层来做出预测，用来确保即使是隐藏单元和中间层也参与了特征计算，即可以进行图片分类

这个Inception网络是由 Google公司的作者所研发的，它被叫做 GoogleLeNet，这个名字是为了向 LeNet网络致敬。

# 第十一部分：目标定位

### （一）图像分类和定位算法

我们之前学习过图像分类，但是如何实现图像中的目标定位呢？

<img src='images/目标定位示例.png' width =600>

在之前的softmax回归中，假设我们要识别3 种目标，会输出一个 4 * 1 的向量，来表示每种目标的概率。     
想要同时输出每种目标的位置，我们可以给每一个元素加上4个数字（变成一个数组）， 分别是 $b_x,b_y,  b_h, b_w \in (0,1)$   
我们约定图片左上角的坐标为(0,0)，右下角标记为(1,1)。方框的中心点表示为 (𝑏𝑥,𝑏𝑦)，边界框的高度为 𝑏ℎ，宽度为 𝑏𝑤。   
比如𝑏𝑥的值是 0.5，表示汽车位于图片水平方向的中间位置； 𝑏𝑦大约是 0.7，表示汽车位于距离图片底部 3/10的位置；    
𝑏ℎ约为 0.3，表示红色方框的高度是图片高度的 0.3倍； 𝑏𝑤约为 0.4，表示红色方框的宽度是图片宽度 的 0.4倍。

所以最后的输出标签 y 应该如下： 
$$
y=\begin{bmatrix}
p_c\\
b_x\\
b_y\\
b_h\\
b_w\\
c_1\\
c_2\\
c_3
\end{bmatrix}

$$

其中$p_c=1$表示图中有目标，$p_c=0$表示图中没有有目标，    
$p_c=1$ 时， $c_1,c_2,c_3$ 中只有一个为1，其他都为0，表示是哪一种目标    
$p_C=0$ 时， 其他参数毫无意义     
该神经网络的损失函数为：  
$y_1=1$ 时   
$$

L(\hat{y},y) =(\hat{y_1}-y_1)^2+(\hat{y_2}-y_2)^2 + …… + (\hat{y_8}-y_8)^2  \\
$$
$y_1=0$ 时 , 
$$
 L(\hat{y},y) =(\hat{y_1}-y_1)^2  \\


$$


**使用神经网络是如何进行目标定位的？**      
在目标定位任务中，可以用一个已经训练好的分类网络（如VGG、ResNet等），并修改最后的全连接层，使其输出中包含边界框的四个参数。    
也就是说，调整最后全连接层的输出即可，不需要做出额外的操作，因为神经网络自己会去学习怎么定位。

**如何衡量算法准确性：**     

我们使用 **交并比函数** 来评价对象检测算法    

<img src='images/交并比函数.png' width=500>   \
<img src='images/交并比.png' width=250>

完美重叠时，IoU为1    
IoU >= 0.5 时 认为检测正确

### （二）特征点识别

上面我们采用的定位点是车的中心点，但是有时我们可以采用一些其他的特征点来定位，     
比如在人脸识别算法中我们想知道眼角的具体位置,我们可以使用特征点 $ ( l_{1x},l_{2x} )   $     
除了眼角，我们还可以得到更多的特征点输出，像下图中的特征点都是眼睛的特征点，根据这些输出来判断眼睛的形状，是睁眼还是眯眼     


<img src='images/眼角特征点.png' height=400>


假设脸部有 64个特征点，有些点甚至可以帮助你定义脸部轮廓或下颌轮廓。此时我们就有了 $ ( l_{1x},l_{2x} ),……,( l_{64x},l_{64x} )   $    
再加上一个1或0表示是否有人脸，我们得到了一个含有129个输出单元的特征 l       

此时我们没有用到宽和高参数，而是用到了很多的特征点来实现定位   

为了构建这样的网络，你需要准备一个标签训练集，也就是图片 𝑥和标签 𝑦的集合，这些点都是人为辛苦标注的。     


注意： 每个特征点在所有图中表示的含义必须一致，比如特征点 1 始终是右眼的外眼角，特征点 2 始终是右眼的内眼角

### （三）滑动窗口目标检测算法

假如你想构建一个汽车检测算法，首先要创建一个标签训练集。训练时可以剪切图片，使整张图片几乎都被汽车占据。

使用这些训练集训练出的神经网络，可以用于识别是否有汽车，也就是输出0或1即可     
然后在一张图片中滑动一个特定大小的窗口，把这个小窗口内的东西输入神经网络 进行预测。     
每次输入给卷积网络的只有窗口内的区域，依次重复操作，直到这个窗口滑过图像的每一个角落。

<img src='images/滑动窗口.png' width=600>

窗口的大小和移动的步幅都是可以调整的

然而因为卷积网络要一个个地处理每个小方块，所以如果你选用的步幅很大，粗糙间隔尺寸可能会影响性能。       
如果采用小粒度或小步幅，传递给卷积网络的小窗口会特别多，计算成本很高     

为了解决计算成本的问题，我们将使用 **滑动窗口的卷积实现（Convolutional implementation of sliding windows）**

为了构建滑动窗口的卷积应用，首先要知道如何把神经网络的全连接层转化成卷积层。    

<img src='images/滑动窗口的卷积实现.png' width=600>

注意：   
1. 这里输出y有 4个数字，它们分别对应行人、汽车、摩托车和背景出现的概率。
2. 这里的前几层是一样的，     
对于第一个全连接层，替换为使用 5 * 5 * 16  的400个过滤器进行卷积操作，输出维度是 1×1×400   
对于第二个全连接层，替换为使用 1 * 1 * 400 的400个过滤器进行卷积操作，输出维度是 1×1×400   
最后的softmax层替换为使用 1 * 1 * 400 的 4个过滤器进行卷积操作，最终得到这个 1×1×4的输出层

掌握了卷积知识，我们再看看如何通过卷积实现滑动窗口对象检测算法。    
假设对每一个滑动窗口都要输入14×14×3的图片，每个滑动窗口最后的输出维度是 1×1×4       
假设测试集的单张图片尺寸是 16×16×3，窗口移动步幅为 2， 那我们可以知道一共会有 4 个窗口    
所以在下图的第二行最后的输出中，我们可以看到 2×2×4 中的 4个格子，每一个格子都对应一个窗口的 1×1×4 输出

<img src='images/滑动窗口的卷积实现2.png' width= 600>

这4次卷积操作中的前几步的神经网络是一样的，所以不用分别为每个窗口去执行前几步的卷积了，大大节省了计算成本    
所以该卷积操作的原理是我们不需要把输入图像分割成四个子集，分别执行前向传播，     
而是把它们作为一张图片输入给卷积网络进行计算，其中的公共区域可以共享很多计算。

注意：    
滑动窗口不能像 图像分类和定位算法 那样输出任意的边框和位置！     
每一个滑动窗口的输出只包括 “是否存在”，不包括 “位置和尺寸”！ 这也是为什么之前说滑动窗口需要很多的小窗口，所以计算成本大。      
每一个窗口的位置和尺寸形状都是预先设定好的！·


### （四）YOLO算法（YOU ONLY LOOK ONCE）

滑动窗口卷积算法 仍然存在一个缺点，就是边界框的位置可能不够准确，不能输出最精准的边界框。    
因此我们使用 **YOLO算法**

**算法流程如下：**    

<img src='images/YOLO算法.png' width=250>

假设输入图像是100×100的，然后在图像上放一个网格。    
这里使用用 3×3 网格，实际实现时会用更精细的网格，比如 19×19。      
然后使用之前介绍的图像分类和定位算法，将算法逐一应用到 9个格子上        
9个格子中的每一个都输出一个8维的标签 𝑦 ，和之前一样，所以总的输出尺寸是 3×3×8
$$
y=\begin{bmatrix}
p_c\\
b_x\\
b_y\\
b_h\\
b_w\\
c_1\\
c_2\\
c_3
\end{bmatrix}

$$

注意：    
1. 如果一个目标跨越了多个格子，把它分配给它中心点所在的格子   
2. 每个方框的左上角是(0,0)，右下角是(1,1)，
3. $b_x,b_y,b_h,b_w$ 是相对于格子尺寸的比例， $b_x,b_y$ 需要在0到1之间，$b_h,b_w$可以大于1

**YOLO算法的优点：**    
1. YOLO算法 和 图像分类和定位算法 一样可以输出 具有任意宽高比的边界框 和 更精确的坐标，   
而 滑动窗口目标检测算法 的边界框的位置和大小都是预先固定的
2. YOLO算法 可以输出多个目标，而 图像分类和定位算法 只能输出一个目标
3. YOLO算法 并不需要在 3×3网格上跑 9次算法，而是使用了一个卷积网络进行单次卷积，有很多共享计算步骤，所以这个算法效率很高。
4. YOLO算法 由于是卷积实现，速度非常快，可以做到实时识别

然而现在我们还会遇到一个问题，某个对象会被检测出多次 而不是 一次。    
比如下图中不止一个格子会认为这辆车中点应该在格子内部，   

<img src='images/重复检测现象.png' width=400>

这个问题可以用 **非极大值抑制** 解决，过程如下：

**非极大值抑制的流程：**      
1. 看看每个格子输出的概率Pc，删掉所有 Pc<0.6 的边界框
2. 在剩余Pc中选取最大的
3. 删除所有 和这个边框有很高交并比的其他边界框 的输出
4. 重复 操作2 和 操作3，直到没有框剩下

非极大值抑制意味着你只输出概率最大的分类结果，但抑制很接近但不是最大的其他预测结果，所以这方法叫做非极大值抑制。

注意：    
刚刚介绍的是检测单个对象的情况，如果你尝试同时检测三个对象，比如说行人、汽车、摩托，那么输出向量就会有三个额外的分量。       
这时候要独立进行三次非极大值抑制，对每个输出类别都做一次

### （五）锚框 (Anchor Boxes)

到目前为止，对象检测中存在的一个问题是每个格子只能检测出一个对象，     
如果想让一个格子检测出多个对象，可以使用 **锚框**

看下面这个例子    
注意行人的中点和汽车的中点几乎在同一个地方，两者都落入到同一个格子中。     
正常来说只能从两个结果中选择一个，而锚框可以输出多个

<img src='images/锚框.png' width=250>

**锚框的过程：**
1. 预先定义几个不同形状的 anchor box，一般至少要 5 个， 这里使用 2 个作为示例    
<img src='images/两个锚框.png' width=300>

2. 定义类别标签，不再是$\begin{bmatrix}
p_c & b_x & b_y & b_h & b_w & c_1 & c_2 & c_3
\end{bmatrix}^T$    
而是 $y = \begin{bmatrix}
p_c & b_x & b_y & b_h & b_w & c_1 & c_2 & c_3 & p_c & b_x & b_y & b_h & b_w & c_1 & c_2 & c_3
\end{bmatrix}^T$,      
也就是重复了两次，前面的8个参数和第一个框关联，后面的8个参数和第二个锚框关联     
由于行人更类似于第一个锚框的形状，车子更类似于第二个锚框的形状，     
所以可以用前8个参数来表示行人，后八个参数来表示车子

用anchor box之前，3×3网格的输出 𝑦 是 3×3×8。这里使用了两个锚框，输出 𝑦 是 3×3×(8×2)

注意：        
如果你有两个anchor box，但在同一个格子中有三个对象，这种情况算法处理不好，    
还有一种情况是，两个对象都分配到一个格子中，而且它们的 anchor box形状也一样，这也是算法处理不好的     
但是这些其实出现不多，所以对性能的影响应该不会很大。

人们一般手工指定 anchor box形状，你可以选择 5到 10个 anchor box形状，覆盖到多种不同的形状·

在锚框于YOLO算法结合时，如果使用 3*3 的格子 和 2 个锚框，那么 9 个格子中任何一个都会有两个预测的边界框，所以要进行非极大值抑制     

<img src='images/锚框+YOLO.png' width=600>

最后，如果你有三个对象检测类别(行人，汽车和摩托车)，那么要对 每个类别 单独运行非极大值抑制

# 第十二部分：人脸识别和神经风格转换（Face recognition & Neural style transfer）

### （一）人脸识别

在人脸识别的相关文献中，人们经常提到人脸验证（face verification）和人脸识别 face recognition）。    
**人脸验证问题：** 如果你有一张输入图片，以及某人的ID或者是名字，要做的是验证输入图片是否是这个人。有时候也被称作 1对 1问题。      
**人脸识别问题：** 难度大很多，因为容错率低很多。

同时，一个重大问题是每个人的照片只有一张。如果用softmax单元的输出来对应每个员工，如此小的训练集得到的效果并不好，    
同时每新增一个员工，就要调整网络结构和重新训练，明显不可行     

因此，下面我们要介绍 **一次学习（One-shot learning）** 的办法

#### (1) Similarity函数 和 Siamese 网络  


我们要神经网络以两张图片作为输入，然后输出这两张图片的差异值。     
也就是要学习 Similarity函数 𝑑(𝑖𝑚𝑔1,𝑖𝑚𝑔2)=𝑑𝑒𝑔𝑟𝑒𝑒 𝑜𝑓 𝑑𝑖𝑓𝑓𝑒𝑟𝑒𝑛𝑐𝑒 𝑏𝑒𝑡𝑤𝑒𝑒𝑛 𝑖𝑚𝑎𝑔𝑒𝑠，     
 
我们设置一个超参数 τ    
𝑑(𝑖𝑚𝑔1,𝑖𝑚𝑔2) < τ，认为两张图片是同一个人，     
𝑑(𝑖𝑚𝑔1,𝑖𝑚𝑔2) > τ，认为两张图片是不同的两个人

Similarity函数 的功能具体怎么实现呢？ 可以使用 **Siamese网络**    

<img src='images/Siamese网络.png' width=600>   
 
之前我们都是把图像经过一系列处理得到的 特征向量 放入 softmax层 进行分类，但在这里我们不这么做         
我们把这个 特征向量 叫做 𝑓(𝑥(1))。你可以把 𝑓(𝑥(1))看作是输入图像 𝑥(1)的编码        
把第二张图片输入到 同一个神经网络，得到另一个 特征向量 𝑓(𝑥(2))，这个向量代表第二个图片的编码     
这里 𝑥(1)和 𝑥(2)仅仅代表两个输入图片，他们没必要非是第一个和第二个训练样本，可以是任意两个图片。

然后就可以计算d：
$$
d(x^{(1)}, x^{(2)}) = \|f(x^{(1)}) - f(x^{(2)})\|_2^2

$$

对于两个不同的输入，运行相同的卷积神经网络，然后比较它们，这就是Siamese网络架构。
 
注意：这两个网络有相同的参数，所以实际要做的就是训练一个网络

那如何训练定义损失函数来训练这个网络呢？ 我们需要用到 **三元组损失函数** 来计算 **损失（ Triplet 损失）**, 进而实现梯度下降

我们一般需要比较  3张图片，Anchor图片、 Positive图片和 Negative图片，简写成 𝐴、 𝑃、 𝑁       
让 Anchor图片和 Positive图片（ Positive意味着是同一个人）的距离很近，    
Anchor图片与 Negative图片（ Negative意味着是非同一个人）的距离很远， 


<img src='images/三元组损失.png' width=600>

我们最后想要的是网络的参数或者编码能够满足以下特性：    
$$
d(A,P)   + \alpha \leq  d(A,N)
$$
即 

$$
d(A,P) - d(A,N)  + \alpha \leq 0   
$$
即 
$$
\|f(A) - f(P)\|_2^2 -\|f(A) - f(N)\|_2^2 + \alpha \leq 0
$$

其中 $\alpha$ 超参数 是防止把所有东西都学成0，出现 0-0<0 的无效情况。这个参数也叫 间隔（margin）

接下来就可以得到损失函数了：     
$$
L(A,P,N)  = max( \|f(A) - f(P)\|_2^2 -\|f(A) - f(N)\|_2^2 + \alpha , 0)
$$

只要前面一项小于0了，那么损失函数就是0

这是一个三元组定义的损失函数，整个网络的代价函数为：   

$$
I = \sum_{i=1}^{m} L(A^{(i)},P^{(i)},N^{(i)})
$$

假如你有一个 10000个图片的训练集，里面是 1000个不同的人的照片，    
你要做的就是取这 10000个图片，然后生成这样的三元组，然后训练你的学习算法，对这种代价函数用梯度下降.    

注意：    
1. 由于需要成对的 𝐴 和 𝑃，即同一个人的成对的图片，因此对于训练集，你需要确保有同一个人的多个图片    
当然，训练完这个系统之后，你可以应用到你的一次学习问题上，只需要一张照片即可。
2. 如果你从训练集中随机地选择 𝐴、 𝑃和 𝑁，其实 $d(A,P) + \alpha \leq  d(A,N)$ 这个条件很好达到，因此网络不能从中学到什么。    
所以要尽可能选择难训练的三元组𝐴、 𝑃和 𝑁。所谓的 “难训练” 是指 d(A,P) 很接近 d(A,N)。

现在的人脸识别系统，尤其是大规模的商业人脸识别系统都是在很大的数据集上训练，这些数据集并不容易获得。            
幸运的是，一些公司已经训练了这些大型的网络并且上传了模型参数。      
所以一般下载别人的预训练模型，而不是一切都要从头开始。

#### （2）人脸验证与二分类

除了Triplet loss，还有其他学习参数的方法，比如把人脸识别当成一个二分类问题

如下图，Siamese网络不变，但是把特征向量 输入到逻辑回归单元，如果是相同的人，那么输出是 1，若是不同的人，输出是 0。

<img src='images/人脸验证和二分类.png' width=600>

这里的逻辑回归单元具体实现如下：         
$$
\hat{y} = \sigma\left(\sum_{k=1}^{128} w_{i} |f(x^{(i)})_k - f(x^{(j)})_k| + b\right)

$$

其中   
$f(x^{(i)})_k$ 代表图片 $x^{(i)}$ 的编码，下标 𝑘代表选择这个向量中的第 𝑘个元素，    
$ |f(x^{(i)})_k - f(x^{(j)})_k| $ 对这两个编码取元素差的绝对值。这一块可以替换为 $\frac{(f(x^{(i)})_k - f(x^{(j)})_k)^2}{f(x^{(i)})_k + f(x^{(j)})_k}$ , 这个公式也叫做 $X^2$ 公式, 也被称为 X 平方相似度

最后的逻辑回归增加了参数 𝑤𝑖和 𝑏，就像普通的逻辑回归一样。    
与之前类似，你正在训练一个Siamese网络，意味着上面这个神经网络拥有的参数和下面神经网络的相同

注意：    
1. 对于一张已经在数据库中的图片，不需要每次都重新计算特征向量，可以先计算好存起来，要比较新照片时就直接拿出来用
2. 和之前不同，这里使用成对图片的训练集， 而不是3个一组。目标标签是 1表示一对图片是一个人，目标标签是 0表示图片中是不同的人。

### （二）神经风格迁移(neural style transfer)

下图所示的就是一个神经风格迁移案例        
我们使用 𝐶 来表示内容图像， 𝑆 表示风格图像， 𝐺 表示生成的图像。

<img src='images/神经风格迁移.png' width=300>

在深入了解如何实现神经风格迁移之前，我们先直观地介绍 卷积神经网络不同层之间的具体运算

#### （1）卷积网络的原理 探究过程

我们之前就了解过，神经网络的浅层专注于局部特征，越深获得的特征越复杂、越完整      
这里对这个研究过程进行详细的展开：


**在神经网络中，一个“隐藏单元”指的是隐藏层中的一个神经元**


**查看第一层的输出：**
1. 训练网络：首先，你训练了一个如AlexNet这样的卷积神经网络，这是一个深层的网络，通常包含多个卷积层和池化层。
2. 遍历训练集：经过训练后，你通过网络传递整个训练集，监控每一层的输出，以便于分析每个隐藏单元（神经元）的激活情况。
3. 最大化激活的图片块：
对于每一层的每个隐藏单元，你查找那些使得这个单元激活值最大的图片或图片的一部分（图片块）。这些图片块是那些在经过卷积层后，能够在特定神经元产生最强响应的输入区域。     
例如，第一层可能专注于检测边缘、颜色或纹理等简单特征。通过找出哪些图片块最能激活某个单元，可以看出这个单元在学习过程中专门对哪些视觉特征作出响应。    
4. 局部响应特性：在网络的早期层（如第一层），每个隐藏单元只能“看到”输入图片的一小部分。这是因为第一层的卷积操作通常只覆盖输入图片的一个小局部区域。
5. 可视化激活：通过选取那些最大化特定单元激活的图片块，你可以可视化这些单元的功能。例如，如果一个单元在被图片中的垂直边缘激活时活跃度最高，那么这表明该单元可能在检测垂直方向的边缘。

然后你可以选一个另一个第一层的隐藏单元，重复刚才的步骤    
在更深的层上，你可以重复这个过程。

#### (二) 神经风格迁移

我们的目标是：给你一个内容图像𝐶 和一个风格图片 𝑆，生成一个新图片 𝐺      
然而 𝐺 不是一步就生成的，而是先去 生成一个随机噪点图片，然后给 生成的图像 定义一个代价函数，     
通过不断梯度下降来最小化代价函数，从而实现生成图片的逐渐渲染成型

因此我们首先要为生成的图像定义一个代价函数，用来评判某个生成图像的好坏



我们把这个代价函数定义为两个部分         
**内容代价 $J_{content}(C,G)$  :**  度量生成图片 𝐺的内容与内容图片 𝐶 的内容有多相似。     
**风格代价 $J_{style}(S,G)$  :**  度量生成图片 𝐺的风格和风格图片 𝑆 的风格的相似度。   

最后：$J(G)=\alpha J_{content}(C,G) + \beta J_{style}(S,G)$


**内容代价函数的具体算法：**

1. 选择一个中间层 $l$
2. 取 $a^{[l][C]}$ 和 $a^{[l][G]}$，代表这两个图片 𝐶 和 𝐺 的 𝑙 层的激活函数值。如果这两个激活值相似，那么就意味着两个图片的内容相似。
3. 使用定义 (L2范数的平方)：
$$
J_{\text{content}}(C, G) = \frac{1}{2} \left\| a^{[l][C]} - a^{[l][G]} \right\|^2
$$


注意：     
如果 𝑙 是个很小的数，比如用隐含层 1，由于进行了过于底层特征的比较，你的生成图片会过于接近你的内容图片。     
如果 𝑙 是个很大的数，由于层数太深，那就会太抽象，比如只会确保生成图片里有一个狗即可，与内容图片差距过大     
所以在实际中，通常 𝑙 会选择在网络的中间层，既不太浅也不很深，    

**风格代价函数的具体算法：**     

我们首先要搞清楚，什么是 “图片的风格”   
在下图中，我们选择了某一层 $l$, 这一层的尺寸为 $n_h \times n_w \times n_c$

<img src='images/风格的意义.png' width=600>


图片的风格 就是 𝑙 层中各个通道之间激活项的相关系数。什么是 不同通道之间激活项的相关系数呢？   
为了理解 “激活项的相关系数”，我们把这个激活块的不同通道渲染成不同的颜色。      
假如我们有 5个通道，我们将它们染成了五种颜色     

<img src='images/激活项的相关系数.png' width=400>

我们知道每个通道对应上一层隐藏层中的一个 过滤器，也就是检测对应的某一个特征      
假设上一层输出的左下角是黄色条纹，第一个通道检测的是黄色特征，因此被激活，第二个通道检测的是条纹特征，因此也被激活        
换句话说，上一层激活值的左下角在第一个通道中含有某个激活项，同时 第二个通道 在这个位置也含有某个激活项，          
如果这幅图片中出现条纹的地方很大概率是黄色的，就认为这两个通道有高度相关性        
而相关系数描述的就是当图片某处出现这种条纹时，该处又同时是黄色的可能性。    


如果上一层激活值的左下角在第一个通道中含有某个激活项，同时 第二个通道 在这个位置也含有某个激活项，于是它们组成了一对数字            
然后我们再看看这个激活项块中其他位置的激活项，它们也分别组成了很多对数字，分别来自各个通道        
现在我们得到了很多个数字对，当我们取得这两个 𝑛𝐻×𝑛𝑊 的通道中所有的数字对后，就可以计算它们的相关系数了


计算过程如下：    
1. 计算风格矩阵 $G^{[l](S)}$ , 𝑙 表示层数， 𝑆 表示风格图像 。$ G^{[l](S)}$ 的尺寸为$ n_c^{[l]}\times n_c^{[l]}$
$$
G^{[l](S)}_{kk'} = \sum_{i=1}^{n_H^{[l]}} \sum_{j=1}^{n_W^{[l]}} a^{[l](S)}_{i,j,k} a^{[l](S)}_{i,j,k'}
$$
$$
a_{i,j,k}^{[l]} = 隐藏层 l 中 (i,j,k)位置的激活项 , 𝑖、 𝑗、 𝑘分别代表该位置的高度、宽度以及对应的通道数\\
$$
在这个矩阵中 𝑘和 𝑘′元素被用来描述 𝑘通道和 𝑘′通道之间的相关系数。   
$G^{[l]}_{kk'}$ 可以用来测量 𝑘 通道与 𝑘′ 通道中的激活项之间的相关系数， 𝑘 和 𝑘′会在 1到 $n_c$之间取值， $n_c$ 就是 𝑙 层中通道的总数量。

2. 同样，我们也对生成的图像进行这个操作
$$
G^{[l](G)}_{kk'} = \sum_{i=1}^{n_H^{[l]}} \sum_{j=1}^{n_W^{[l]}} a^{[l](G)}_{i,j,k} a^{[l](G)}_{i,j,k'}
$$
上标 (𝑆)和 (𝐺)分别表示在风格图像 S中的激活项和在生成图像 𝐺的激活项。     


3. 要注意，如果两个通道中的激活项数值都很大，那么 $G^{[l]}_{kk'}$ 也会很大，对应强相关。反之，不相关，$G^{[l]}_{kk'}$就会很小
4. 现在我们有2个风格矩阵，分别从风格图像 𝑆 和生成图像 𝐺 得到。

<img src='images/风格矩阵计算.png' width=600>

5. 最后，如果我们将𝑆和 𝐺代入到风格代价函数中去计算，
$$
J_{\text{style}}(S, G) = \frac{1}{(2n_H^{[l]} n_W^{[l]} n_C^{[l]})^2} \sum_{k} \sum_{k'} \left( G^{[l](S)}_{kk'} - G^{[l](G)}_{kk'} \right)
$$
即
$$
J_{\text{style}}(S, G) = \frac{1}{\left(2n_H^{[l]} n_W^{[l]} n_C^{[l]}\right)^2} \left\|G^{[l](S)} - G^{[l](G)}\right\|_F^2

$$
注意：
1. 我们之所以用大写字母 𝐺来代表这些风格矩阵，是因为在线性代数中这种矩阵有时也叫 Gram矩阵，但在这里我只把它们叫做风格矩阵。
2. 如果你对各层都使用风格代价函数，会让结果变得更好。如果要对各层都使用风格代价函数，你可以把各个层的结果（各层的风格代价函数）都加起来，并对每个层定义权重𝜆[𝑙]，这样将使你能够在神经网络中使用不同的层，使得我们的神经网络在计算风格时能够同时考虑到低级和高级特征的相关数。

**算法的整个流程如下：**    
1. 随机初始化生成图像 𝐺，它可以是任何你想要的尺寸。
2. 梯度下降 来减小代价函数 J(G), 更新 $G := G - \frac{\partial}{\partial G} J(G)$ 。在这个步骤中，你实际上更新的是图像 𝐺的像素值。




<img src='images/神经迁移代价函数更新.png' width=600>

### （三）1维到3维的推广

我们已经学习了许多关于卷积神经网络（ConvNets）的知识，从卷积神经网络框架，到如何使用它进行图像识别、对象检测、人脸识别与神经网络转换。    
即使我们大部分讨论的图像数据都是 2D数据，但许多思想不仅局限于 2D图像，甚至可以延伸至 1D，乃至 3D数据。

**一维卷积：**

在这种情况下你需要使用一个 1维过滤器 进行卷积，你只需要一个 1×5的过滤器，在不同的位置中应用类似的方法

<img src='images/一维卷积.png' width=500>  \
<img src='images/一维卷积流程.png' width=500>

当你对这个1维信号使用卷积，你将发现一个 14维的数据与 5维数据进行卷积，并产生一个 10维输出。    
如果你有 16个过滤器，可能你最后会获得一个 10×16的数据，这可能会是你卷积网络中的某一层。

不过对于1D数据而言，我们后面还会学到序列模型（RNN），专门解决这种问题

**三维卷积：**

比如CT的人体切片。    
也要使用 3D 的过滤器进行卷积，原理和上面相同

# 第十三部分：序列模型（Sequence Models）

### （一）序列模型的基础介绍

序列模型有很多，包括 **循环神经网络（RNN）** 等      

序列模型有很多应用场景：    
**语音识别:** 输入音频片段𝑥，输出对应的文字记录 𝑦。输入和输出数据都是序列模型，因为 𝑥是一个按时播放的音频片段，输出 𝑦是一系列单词。      
**音乐生成:** 只有输出数据𝑦是序列，而输入数据可以是空集 或一个整数，这个数可能指代音乐风格,也可能是头几个音符      
还有 DNA序列分析，机器翻译，视频动作识别，情感分类

<img src='images/序列模型案例.png' width=500>


假如你想要建立一个能够自动识别句中人名位置的序列模型，那么这就是一个 **命名实体识别问题**，这常用于搜索引擎        

假设输入的一句话：“Harry Potter and Herminoe Granger invented a new spell.”         
这是9个单词组成的序列，分别对应 $x^{<1>},x^{<2>},……, x^{<9>}$，中间的某一项用 $x^{<t>}$ 表示，输入序列长度是 $T_x = 9$        
输出数据是 $y^{<1>},y^{<2>},……, y^{<9>}$, 输出序列长度是 $T_y = 9$     

$x^{(i)<t>}$ 表示第 i 个训练样本的第 t 个输入元素，$y^{(i)<t>}$ 表示第 i 个训练样本的第 t 个输出元素       
$T_x^{(i)}$ 代表第 i 个训练样本的输入序列长度，$T_y^{(i)}$ 代表第 i 个训练样本的输出序列长度

想要表示一个单词，首先要做一张词表，包含从'a','abandon',……,'zulu' 的所有词        
$x^{<t>}$ 就是一个和这张词表尺寸一样的向量，这个单词的位置是1，剩余位置都是0， 这称为 **one-hot表示法**     
如果你的词典大小是 10000的话，那么这里的每个向量 $x^{<t>}$ 都是 10,000维的 

注意：    
1. 如果遇到了一个不在你词表中的单词，要创建一个新的标记 \<UNK>，来表示不在词表中的单词
2. 对于商业应用来说，或者对于一般规模的商业应用来说 30,000到 50,000词 的词典比较常见

### (二) 循环神经网络模型（RNN）

由于单个 $x^{<t>}$ 的体量太大，同时输入和输出数据的长度 在不同例子中不同，所以普通的标准神经网络不适用，     
而且标准神经网络并不共享从文本的不同位置上学到的特征。    
比如假设神经网络已经学习到了在位置 1出现的 Harry可能是人名，那么如果Harry出现在其他位置，它应该也要能够自动识别其为人名。    
这类似于卷积神经网络能将部分图片里学到的内容快速推广到图片的其他部分，而我们希望对序列数据也有相似的效果。

因此需要使用 **循环神经网络RNN**


<img src='images/RNN.png' width=600>


步骤如下：   
1. 首先在零时刻需要构造一个激活值$a^{<0>}$，这通常是零向量。
2. 将第一个词输入一个神经网络层，经过第一个神经网络的隐藏层，可以得到预测输出，判断这是否是人名的一部分
3. 当网络读到第二个单词 $x^{<2>}$ 时，它不是只用 $x^{<2>}$ 就预测出 $y^{<2>}$，而是结合上一步的激活值 $a^{<1>}$。
4. 如此循环直到最后一个时间步


每一步的计算过程如下：     

$$
a^{<t>} = g_1(W_{aa} a^{<t-1>} + W_{ax} x^{<t>} + b_a)
$$
$$
\hat{y}^{<t>} = g_2(W_{ya} a^{<t>} + b_y)
$$
我们可以将这里的符号简化一下：    
$$
a^{<t>} = g_1(W_{a}[a^{<t-1>},x^{<t>}] + b_a)
$$
$$
\hat{y}^{<t>} = g_2(W_{y} a^{<t>} + b_y)
$$
其中：
$$
[W_{aa} : W_{ax}] = W_a \\

[a^{<t-1>},x^{<t>}] 表示
\begin{bmatrix}
a^{<t-1>} \\
x^{<t>}
\end{bmatrix}


$$
如果 𝑎是 100维的，𝑥是 10,000维的，那么 𝑊𝑎𝑎就是个 100×100 维的矩阵， 𝑊𝑎𝑥就是个 100×10000 维的矩阵，𝑊𝑎就会是个 100×10000 维的矩阵。

注意：    
1. 循环神经网络是从左向右扫描数据，同时每个时间步的参数也是共享的。    
从 $x^{<1>}$ 到隐藏层之间 的一系列参数用 $𝑊_{ax}$ 来表示，每个时间步使用的都是相同的参数$𝑊_{ax}$。         
时间步之间 激活值传递 的一系列参数用 $𝑊_{aa}$ 来表示，每个时间步使用的都是相同的参数$𝑊_{aa}$    
隐藏层和输出结果之间 的一系列参数用 $𝑊_{ya}$ 来表示，每个时间步使用的都是相同的参数$𝑊_{ya}$
2. $W_{ax}$ 的第二个下标x意味着要乘以某个x类型的变量，第一个下标a表示它是用来计算某个a类型的变量，同理推广
3. 循环神经网络用的激活函数经常是tanh，不过有时候也会用 ReLU。     
如果y输出类型是 二分问题，用 sigmoid函数作为激活函数，（这里采用）    
如果y输出类型是 𝑘类别分类问题，用 softmax作为激活函数





**向前传播流程如下**   

<img src='images/RNN向前传播.png' width=600>

某个时间步𝑡上某个单词的 预测值的 损失函数。：   
$$
L^{<t>}(\hat{y}^{<t>}, y^{<t>}) = -y^{<t>} \log(\hat{y}^{<t>}) - (1 - y^{<t>}) \log(1 - \hat{y}^{<t>})
$$
它对应的是序列中一个具体的词，如果它是某个人的名字，那么𝑦<𝑡>的值就是 1，然后神经网络将输出这个词是名字的概率值，比如 0.1     
这是 **标准逻辑回归损失函数**，也叫 **交叉熵损失函数（ Cross Entropy Loss）**，它和之前我们在二分类问题中看到的公式很像。

**整个序列的损失函数：**
$$
L(\hat{y}, y) = \sum_{t=1}^{T_x} L^{<t>}(\hat{y}^{<t>}, y^{<t>})

$$
    
**反向传播的计算过程如下：**（框架会自动计算，所以仅供了解）   

<img src='images/RNN反向传播.png' width=600>

这个算法有一个很别致的名字，叫做 **通过（穿越）时间反向传播 backpropagation through time）**


我们现在看到的RNN结构中 输出尺寸 $T_x$ 等于 输出尺寸 $T_y$, 然而当他们不相等时，RNN结构要做出改变        
当输入序列有很多的输入，输出序列有很多的输出时，称为 多对多 （many-to-many）的结构，比如刚刚的人名识别和机器翻译     
当输入序列有很多的输入，输出序列只有一个数字时，称为 多对一 （many-to-one）结构，比如句子情感识别   
当输入序列只有一个数字，输出序列只有一个数字时，称为 一对一 （one-to-one）结构，这个不适合RNN，适合之前讲的网络      
当输入序列只有一个数字，输出序列有很多的输出时，称为 一对多 （one-to-many）结构，比如音乐生成   



<img src='images/各种RNN类型.png' width=600>  \
注：one-to-many 那张图中的正方形之间也是有横向箭头的，被挡住了


### （三）用 RNN构建一个语言模型   
所以什么是语言模型呢？   
比如你听到一个句子，“the apple and pear（pair） salad was delicious.”        
我说的是 “the apple and pair”，还是 “the apple and pear” ？（pear和 pair是近音词）。     
这就是语音识别系统要输出的东西，语音识别系统要使用一个语言模型，计算出这两句话各自的可能性。   
比如 P(pair) = 3.2 *10**-13 而 P(pear) = 5.7 *10**-10， 那就是显然是第二种了，因为概率高很多   

所以语言模型所做的就是，输入一个文本序列，它会告诉你某个特定的句子它出现的概率是多少

为了使用RNN建立语言模型，你首先需要一个很大的 **文本语料库（corpus）** 作为训练集        
语料库是自然语言处理的一个专有名词，意思就是很长的或者说数量众多的句子组成的文本。



假如说，你在训练集中得到这么一句话：“Cats average 15 hours of sleep a day.”     
训练过程如下：     

<img src='images/语言模型.png' width=600>

1. **标记化：** 建立一个字典，然后将每个单词都转换成对应的 one-hot向量   
你可以自行决定要不要把标点符号看成标记，在本例中 ，我们忽略了标点符号，反之需要把标点符号也加入字典
2. 在句尾增加一个额外的标记，叫 \<EOS>，用于定义句子的结尾。  
EOS标记要被附加到训练集中每一个句子的结尾。 注意 \<EOS> 也算入输入输出。
3. 遇到未知的词时，把原词替换为 \<UNK>，我们只针对  \<UNK> 建立概率 模型，而不是针对这个具体的词 
4. 在第 0 个时间步，你要计算激活项𝑎<1>，它是以 𝑥<1>作为输入的函数，而 𝑥<1>是 0向量
5. $a^{<1>}$ 会通过 softmax 来预测第一个词的各种概率，其结果就是 $\hat y^{<1>}$    
 $\hat y^{<1>}$ 是个 10002 维向量，包含了各个词的概率，其中包括字典中的 10000个词 以及\<UNK>，\<EOS>
6. RNN进入下个时间步，使用激活项 𝑎<1> 和正确的第一个词 $\hat y^{<1>}$ 计算出第二个词 $\hat y^{<2>}$，这就是为什么 𝑦<1>=𝑥<2>
7. 如此循环，始终保持 $ x^{<t>}  = \hat y^{<t-1>}$
8. 最后在第9个时间步，然后把 𝑎<9> 和 𝑥<9>（也就是 𝑦<8>）传给它， 会输出 𝑦<9>，理想情况下 最后的得到结果会是\<EOS>，因此停止
9. 为了训练这个网络，需要定义 某个时间步t 的代价函数
$$
L^{<t>}(\hat{y}^{<t>}, y^{<t>}) = -\sum_i y_i^{<t>} \log \hat{y}_i^{<t>}
$$
总体损失函数
$$
L = \sum_{t} L^{<t>}(\hat{y}^{<t>}, y^{<t>})
$$
10. 更细参数，梯度下降


所以RNN中的每一步都会考虑前面得到的单词，比如给它前 3个单词，让它给出下个词的分布，这就是 RNN如何学习从左往右地每次预测一个词。

 
现在有一个新句子，它只包含 3个词 $y^{<1>},y^{<2>},y^{<3>}$，现在 要计算出整个句子中各个单词的概率，     
第一个softmax层会告诉你 𝑦<1>的概率，这也是第一个输出，   
第二个 softmax层会告诉你在考虑 𝑦<1>的情况下 𝑦<2>的概率    
第三个 softmax层告诉你在考虑 𝑦<1>和 𝑦<2>的情况下 𝑦<3>的概率         
把这三个概率相乘，最后得到这个含 3个词的整个句子的概率。

**新序列采样（Sampling novel sequences）**     （略）

这种方法用于训练一个序列模型之后，了解这个模型学到了什么     


下图中编号 1所示的网络是经过训练的

<img src='images/新序列采样.png' width=600>

步骤如下：      
1. 输入 𝑥<1>=0，𝑎<0>=0，在第一个时间步经过 softmax层 后输出各个词的概率向量，在这个向量中随机采样作为 $y^{<1>}$。     
不管你在第一个时间步得到的是什么词，都要把它传递到下一个位置作为输入
2. 和训练的流程一样，除了 每个时间步都在 softmax层 的输出中随机采样，输入下一层
3. 一直进行采样直到得到 EOS标识 ，这代表着已经抵达结尾，可以停止采样了。



### （四）双向循环神经网络 BRNN

前面我们讲到了RNN的网络结构，但是存在一个问题，就是它 在某一时刻的预测 仅使用了从序列之前的 输入信息 ，并没有使用序列之后的信息     
而我们知道句子的语境是根据上下文得出的，所以识别的效果不好      
这个问题可以用 **双向循环神经网络BRNN** 来解决

<img src='images/BRNN.png' width=600>

给定一个输入序列 𝑥<1>到 𝑥<4>，这个序列首先计算前向的 𝑎⃗ <1>，然后计算前向的 𝑎⃗ <2>，𝑎⃗ <3>，𝑎⃗ <4>。    
而反向序列从计算 𝑎⃖<4>开始，反向进行，计算反向的 𝑎⃖<3>，𝑎⃖<2>，𝑎⃖<1>     
把所有这些激活值都计算完了就可以计算预测结果了。
$$
\hat{y}^{<t>} = g\left(W_g [𝑎⃗^{<t>}, 𝑎⃖^{<t>}] + b_y\right)
$$

注意：      
1. 这里的正向和反向计算 都属于 正向传播！因为都是为了算激活值。
2. BRNN的缺点就是你需要完整的数据的序列， 你才能预测任意位置。        
比如一个语音识别系统，需要等待这个人说完，获取整个语音才能处理。

### （五）GRU门控单元

训练很深的神经网络时，随着层数的增加，导数有可能指数型的下降或者指数型的增加，也就是 梯度消失或者梯度爆炸。

其中梯度爆炸很明显也很好解决，    
因为指数级大的梯度会让你的参数变得极其大 ，导致数值溢出。你会看到很多 NaN 或者不是数字的情况。        
梯度爆炸可以用 梯度修剪 解决，也就是 当梯度向量大于某个阈值时，缩放梯度向量，保证它不会太大

而 RNN 中的梯度消失问题比较棘手     
举个例子，我们知道英语语法中有三单原则，但是如果 动作主体 离 动词 很远，就不能保证输出第三人称单数的准确性了          
这是因为很深的神经网络，从输出 $\hat y$ 得到的梯度很难传播回去影响靠前层的权重，也就是梯度消失    
所以RNN 很难处理这种长期依赖效应，因此我们使用 **门控循环单元GRU** 解决这个问题


我们之前看到的RNN单元如下    

<img src='images/RNN unit.png' width=400>


整个GRU单元的结构图如下  

<img src='images/GRU unit.png' width=600>




而GRU单元将会有个新的变量称为 𝑐，代表记忆细胞        
在时间 𝑡处，有记忆细胞 $c^{<t>}$，对于GRU而言 $c^{<t>}=a^{<t>}$。        
在每个时间步，我们计算一个候选值 $\tilde{c}^{<t>} $   
$$
\tilde{c}^{<t>} = \tanh(W_c [\Gamma_r * c^{<t-1>}, x^{<t>}] + b_c)
$$
然后由下面的“门” 决定候选值是否替代记忆细胞 $c^{<t>}$ 的值


GRU中还有一个叫“门”的结构 $\Gamma_u,\Gamma_r \in (0,1)$, 下标u代表更新门，下标r代表相关性。     
$\Gamma_r$ 相关门 代表 $c^{<t-1>}$ 跟 $\tilde{c}^{<t>} $ 有多大的相关性。    
$\Gamma_u$ 更新门 决定 候选值 $\tilde{c}^{<t>} $   是否代替 记忆细胞 $c^{<t>}$ 的值
$$
\Gamma_u = \sigma(W_u [c^{<t-1>}, x^{<t>}] + b_u)  \\
\Gamma_r = \sigma(W_r [c^{<t-1>}, x^{<t>}] + b_r)

$$
对于大多数输入， sigmoid函数的输出总是非常接近 0 或  1, 因此  $\Gamma_u$ 在大多数情况下非常接近0或1         
$$
c^{<t>} = \Gamma_u \ast \tilde{c}^{<t>} + (1 - \Gamma_u) \ast c^{<t-1>}

$$
$\Gamma_u = 1$ 时更新，$\Gamma_u = 0$时不更新 


由于存在记忆单元把前面的值传递到了网络的深层，因此可以解决梯度消失问题

注意：     
1. $\Gamma_u$、$c^{<t>}$ 和 $\tilde{c}^{<t>} $ 的维度一样
2. $\Gamma_u$ 不会真的等于 0或者 1，只是很接近

### （六）长短期记忆（LSTM）

除了GRU单元，LSTM单元也可以做到防止梯度消失，建立深度链接的作用，甚至效果更好

对于GRU而言 $c^{<t>}=a^{<t>}$。对LSTM来说则不是       

他们的计算过程对比如下：

<img src='images/GRU vs LSTM.png' width=600>

可见我们去除了 相关门$\Gamma_r$ ,增加了遗忘门 $\Gamma_f$ 和 输出门$\Gamma_0$

其结构如下：    

<img src='images/LSTM向前传播.png' width=600>  

把这些网络连接起来即：


<img src='images/LSTM网络.png' width=600>

反向传播如下（看看就好，框架会自己算的）

<img src='images/LSTM 反向传播1.png'>
<img src='images/LSTM 反向传播2.png'>
<img src='images/LSTM 反向传播3.png'>




### (七) 深层循环神经网络（Deep RNNs）

这是一种结合标准神经网络和 RNN 的构造方式     

结构如下：    

<img src='images/DeepRNN.png' width=600>

我们看一个具体的计算例子：      
$$
a^{[2]<3>} = g\left(W_a^{[2]} [a^{[2]<2>}, a^{[1]<3>}] + b_a^{[2]}\right)

$$


对于标准的神经网络，你可能见过很深的网络，甚至于 100层深，而对于RNN来说，有三层就已经不少了。    
因为时间的维度，Deep RNN会变得相当大